#### This notebook would be used to run the code on reduced dataset which can then offer finer controls over the parameter space.

In [1]:
cd merity

/data/priyansh/new_kranti/merity


In [2]:
import model_slot as merity

In [3]:
cd ..

/data/priyansh/new_kranti


In [4]:
# Torch imports
import torch
import torch.nn as nn
from torch import optim
from torch.utils.data import  DataLoader
from torch.nn.utils.rnn import pad_packed_sequence, pack_sequence 

# Local imports
import data_loader as dl
import auxiliary as aux
import network as net

# Other libs
from tqdm import tqdm
import numpy as np
import collections
import argparse
import pickle
import ujson
import time
import html
import sys
import re

from configs import config_loader as cl
import corechain as cc
import components as com
import utils.tensor_utils as tu
from utils import embeddings_interface as ei
from utils import natural_language_utilities as nlutils

sys.path.append('/data/priyansh/conda/fastai')
from fastai.text import *
import fastai
from fastai import text, core, lm_rnn
from typing import Any, AnyStr, Callable, Collection, Dict, Hashable, Iterator, List, Mapping, NewType, Optional
import matplotlib.pyplot as plt
from functools import partial
import lriters
from utils.goodies import *

ei.__check_prepared__()
torch.manual_seed(42)
np.random.seed(42)

Loading ULMFIT vocab and vectors from disk. Sit Tight.
Label Cache not found. Creating a new one


Traceback (most recent call last):
  File "/data/priyansh/new_kranti/utils/dbpedia_interface.py", line 133, in __init__
    self.labels = pickle.load(open('resources/labels.pickle'))
TypeError: a bytes-like object is required, not 'str'


NOT IMPLEMENTED FUNCTIONALITY NOT IMPLEMENTED FUNCTIONALITY !!!


In [5]:
#setting up device,model name and loss types.
device = torch.device("cuda")
training_model = 'slotptr'
_dataset = 'lcquad'
_train_over_validation = False
pointwise = False
_train_over_validation = False
finetune = False
bidirectional = True

#Loading relations file.
COMMON_DATA_DIR = 'data/data/common'
_dataset_specific_data_dir = 'data/data/%(dataset)s/' % {'dataset': _dataset}
_inv_relations = aux.load_inverse_relation(COMMON_DATA_DIR)
_word_to_id = aux.load_word_list(COMMON_DATA_DIR)

LM_DIR = Path('./data/models/lm/layer3modelfastaidata100_1ques+cc_a/')

NOT IMPLEMENTED FUNCTIONALITY NOT IMPLEMENTED FUNCTIONALITY !!!


In [6]:
if pointwise:
    training_config = 'pointwise'
else:
    training_config = 'pairwise'

if training_model == 'reldet':
    schema = 'reldet'
elif training_model == 'slotptr' or training_model == 'slotptr_common_encoder' or training_model == 'slotptrortho':
    schema = 'slotptr'
elif training_model == 'bilstm_dot_multiencoder':
    schema = 'default'
else:
    schema = 'default'

In [7]:
parameter_dict = cl.corechain_parameters(dataset=_dataset,training_model=training_model,
                                             training_config=training_config,config_file='configs/macros.cfg')

parameter_dict['batch_size'] = 1000
if _dataset == 'lcquad':
    test_every = parameter_dict['test_every']
else:
    test_every = False
validate_every = parameter_dict['validate_every']

In [8]:
data = aux.load_data(_dataset=_dataset, _train_over_validation = _train_over_validation,
                     _parameter_dict=parameter_dict, _relations =  _inv_relations, 
                     _pointwise=pointwise, _device=device,k=-1)

(3498, 25)
(3498, 25)
(3498, 1000, 25)
(500, 25)
(500, 25)
(500, 1000, 25)
(1000, 25)
(1000, 25)
(1000, 1000, 25)


In [9]:
stoi = ei.vocab
itos = {v: k for k, v in stoi.items()}

In [10]:
chunksize = 24000
BOS = 'xbos'  # beginning-of-sentence tag
FLD = 'xfld'  # data field tag
re1 = re.compile(r'  +')
def _fixup_(x):
    x = x.replace('#39;', "'").replace('amp;', '&').replace('#146;', "'").replace(
        'nbsp;', ' ').replace('#36;', '$').replace('\\n', "\n").replace('quot;', "'").replace(
        '<br />', "\n").replace('\\"', '"').replace('<unk>', 'u_n').replace(' @.@ ', '.').replace(
        ' @-@ ', '-').replace('\\', ' \\ ')
    return re1.sub(' ', html.unescape(x))
def _get_texts_(df, n_lbls=1):
    labels = df.iloc[:, range(n_lbls)].values.astype(np.int64)
    texts = f'\n{BOS} {FLD} 1 ' + df[n_lbls].astype(str)
    for i in range(n_lbls + 1, len(df.columns)): texts += f' {FLD} {i-n_lbls} ' + df[i].astype(str)
    texts = list(texts.apply(_fixup_).values)

    tok = text.Tokenizer().proc_all_mp(core.partition_by_cores(texts))
    return tok, list(labels)
def _simple_apply_fixup_(df):
    labels = [0] * df.shape[0]
    texts = f'\n{BOS} {FLD} 1 ' + df.text
    texts = list(texts.apply(_fixup_).values)
    tok = text.Tokenizer().proc_all_mp(core.partition_by_cores(texts))
    return tok, list(labels)   
def get_all(df, n_lbls):
    tok, labels = [], []
    for i, r in enumerate(df):
        print(i)
        tok_, labels_ = _get_texts_(r, n_lbls)
        tok += tok_;
        labels += labels_
    return tok, labels
def convert_cc(cc):
    print(cc)
    new_cc = []
    for i, element in enumerate(cc):
        if element == 0: continue
        if i%2==0:
            new_cc.append(element)
        else:
            new_cc.extend(list(rel_pkl_rev[element][3]))
    new_cc_str = [itos[x] for x in new_cc]
    print(new_cc_str)

rel_pkl = pickle.load(open('data/data/common/relations.pickle', 'rb'))
rel_pkl_rev = {v[0]:[k]+v[1:] for k,v in rel_pkl.items()}

#### Block which preprocesses questions and paths to fit the way ULMFiT is used to having them

In [11]:
keys = [x for x in data.keys() if 'questions' in x]
with Timer() as timer:
    for k in keys:
        v = [ei.vocabularize(_fixup_(f'{BOS} {FLD} 1 '+' '.join([itos[int(x)] for x in y])).split()) for y in data[k]]
        # Pad each thing in v
        new_v = np.zeros((len(v), data[k].shape[1]))
        _maxlen = new_v.shape[1]
        for i, ques in enumerate(v):
            new_v[i, :min(len(ques), _maxlen)] = ques[:min(len(ques), _maxlen)]
        data[k] = new_v
print(f"Time taken to update data {timer.interval}")
# train_loader = cc.load_data(data, parameter_dict, pointwise, schema=schema)

if False:
    keys = [x for x in list(data.keys()) if x.endswith('paths') ]
    with Timer() as timer:
        for key in keys:    
            if 'pos' not in key: 
                for i, batch in enumerate(data[key]):
                    v = np.array([np.array(ei.vocabularize(_fixup_(f'{BOS} {FLD} 1 '+' '.join([itos[x] for x in cch])).split())[:batch.shape[1]]) for cch in batch])
                    data[key][i] = v

            else:
                v = np.array([np.array(ei.vocabularize(_fixup_(f'{BOS} {FLD} 1 '+' '.join([itos[x] for x in cch])).split())[:data[key].shape[1]]) for cch in data[key]])
                data[key] = v
        #         print(data[key].shape)
    print(f"Time taken to update data {timer.interval}")

Time taken to update data 0.20462914602831006


#### Block which runs the train loader enough times to get it working normally (for pointwise training)

In [12]:
train_loader = cc.load_data(data, parameter_dict, pointwise, schema=schema)
for i, _ in enumerate(train_loader):
    pass
print(i)
for i, _ in enumerate(train_loader):
    pass
print(i)
for i, _ in enumerate(train_loader):
    pass
print(i)

(3498, 25)
349
349
349


### Copy pasting the RNN core module from the fastai

In [13]:
# import warnings
# from fastai.imports import *
# from fastai.torch_imports import *
# from fastai.rnn_reg import LockedDropout,WeightDrop,EmbeddingDropout
# from fastai.model import Stepper
# from fastai.core import set_grad_enabled
# import collections

# from torch import Tensor
# from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence, pack_sequence
# from typing import Sequence, Tuple, TypeVar, Union
# from .imports.core import *
# from fastai.imports.core import *
# IS_TORCH_04 = LooseVersion(torch.__version__) >= LooseVersion('0.4')

# def seq2seq_reg(output, xtra, loss, alpha=0, beta=0):
#     hs,dropped_hs = xtra
#     if alpha:  # Activation Regularization
#         loss = loss + (alpha * dropped_hs[-1].pow(2).mean()).sum()
#     if beta:   # Temporal Activation Regularization (slowness)
#         h = hs[-1]
#         if len(h)>1: loss = loss + (beta * (h[1:] - h[:-1]).pow(2).mean()).sum()
#     return loss

# def repackage_var(h):
#     """Wraps h in new Variables, to detach them from their history."""
#     if IS_TORCH_04: return h.detach() if type(h) == torch.Tensor else tuple(repackage_var(v) for v in h)
#     else: return Variable(h.data) if type(h) == Variable else tuple(repackage_var(v) for v in h)

In [14]:
class BiLstmDot_ulmfit(net.Model):

    def __init__(self, _parameter_dict, _word_to_id, _device, _pointwise=False, _debug=False):

        self.debug = _debug
        self.parameter_dict = _parameter_dict
        self.device = _device
        self.pointwise = _pointwise
        self.word_to_id = _word_to_id

        if self.debug:
            print("Init Models")

        # Load the pre-trained model
#         pretrained_weights = torch.load('./resources/unsup_model_enc.torch', map_location= lambda storage, loc: storage)
#         pretrained_weights = torch.load('./resources/pretr_model_enc.torch', map_location= lambda storage, loc: storage)
        pretrained_weights = torch.load('./ulmfit/wt103/fwd_wt103_enc.h5', map_location= lambda storage, loc: storage)
#         pretrained_weights = torch.load(LM_DIR /'unsup_model_enc.torch', map_location= lambda storage, loc: storage)

        new_vectors = self.parameter_dict['vectors']
        pretrained_weights['encoder.weight'] = T(new_vectors)
        pretrained_weights.pop('encoder_with_dropout.embed.weight')
#         pretrained_weights['encoder_with_dropout.embed.weight'] = T(np.copy(new_vectors))

        # self.encoder = fastai.old.lm_rnn.RNN_Encoder(ntoken=new_vectors.shape[0], emb_sz=400, n_hid=1150, n_layers=1, pad_token=0,qrnn=False).to(self.device)
        # self.encoder.load_state_dict(pretrained_weights)
        # fastai.RNNTrainer
        # self.encoder = com.NotSuchABetterEncoder(
        #     number_of_layer=self.parameter_dict['number_of_layer'],
        #     bidirectional=self.parameter_dict['bidirectional'],
        #     embedding_dim=self.parameter_dict['embedding_dim'],
        #     max_length = self.parameter_dict['max_length'],
        #     hidden_dim=self.parameter_dict['hidden_size'],
        #     vocab_size=self.parameter_dict['vocab_size'],
        #     dropout=self.parameter_dict['dropout'],
        #     vectors=self.parameter_dict['vectors'],
        #     enable_layer_norm=False,
        #     mode = 'LSTM',
        #     debug = self.debug).to(self.device)

        # self.encoder = RNN_Encoder(ntoken=self.parameter_dict['vectors'].shape[0], emb_sz=400,
        #                            n_hid=512, n_layers=1, pad_token=0, qrnn=False)
  
        # Merity
        self.encoder = merity.RNNModel(rnn_type='LSTM', 
                ntoken=self.parameter_dict['vectors'].shape[0],
                ninp=400,
                nhid=1150,
                nlayers=3,
                dropout=0.05,
                dropouth=0.2,
                dropouti=0.2,
                dropoute=0.05,
                wdrop=0,
                tie_weights=True)
        self.encoder.encoder.padding_idx = 0   
        
        
#         self.encoder_path = NotSuchABetterEncoder(
#             number_of_layer=self.parameter_dict['number_of_layer'],
#             bidirectional=self.parameter_dict['bidirectional'],
#             embedding_dim=self.parameter_dict['embedding_dim'],
#             max_length = self.parameter_dict['max_length'],
#             hidden_dim=400,
#             vocab_size=self.parameter_dict['vocab_size'],
#             dropout=self.parameter_dict['dropout'],
#             vectors=self.parameter_dict['vectors'],
#             enable_layer_norm=False,
#             mode = 'LSTM',
#             debug = self.debug).to(self.device)
        
        key_mapping = {
                    'encoder.weight' : 'encoder.weight',
                    'rnns.0.module.weight_ih_l0' : 'rnns.0.weight_ih_l0', 
                    'rnns.0.module.bias_ih_l0' : 'rnns.0.bias_ih_l0', 
                    'rnns.0.module.bias_hh_l0' : 'rnns.0.bias_hh_l0', 
                    'rnns.0.module.weight_hh_l0_raw' : 'rnns.0.weight_hh_l0',
                    'rnns.1.module.weight_ih_l0' : 'rnns.1.weight_ih_l0', 
                    'rnns.1.module.bias_ih_l0' : 'rnns.1.bias_ih_l0', 
                    'rnns.1.module.bias_hh_l0': 'rnns.1.bias_hh_l0', 
                    'rnns.1.module.weight_hh_l0_raw' : 'rnns.1.weight_hh_l0', 
                    'rnns.2.module.weight_ih_l0' : 'rnns.2.weight_ih_l0', 
                    'rnns.2.module.bias_ih_l0' : 'rnns.2.bias_ih_l0', 
                    'rnns.2.module.bias_hh_l0': 'rnns.2.bias_hh_l0', 
                    'rnns.2.module.weight_hh_l0_raw' : 'rnns.2.weight_hh_l0',
                }
        
        for k,v in key_mapping.items():
            pretrained_weights[v] = pretrained_weights.pop(k)

        self.encoder.load_state_dict(pretrained_weights)
#         self.encoder.encoder = nn.Embedding.from_pretrained(torch.FloatTensor(self.parameter_dict['vectors']))    
        
#         self.encoder_path.encoder = nn.Embedding.from_pretrained(torch.FloatTensor(self.parameter_dict['vectors']))
#         self.encoder = self.encoder.to(self.device)
#         self.encoder_path = self.encoder_path.to(self.device)
        
#         self.encoder = RNN_Encoder_Masks(vocab_sz=self.parameter_dict['vectors'].shape[0], emb_sz=400, weight_p=0,
#                                                  n_hid=256, n_layers=1, pad_token=0, qrnn=False).to(device)
#         self.encoder.load_state_dict(pretrained_weights)
#         self.encoder.encoder.weight = torch.tensor(self.parameter_dict['vectors'],device=self.device)
                
#         self.classifier = PoolingLinearClassifier(layers=[400*3, 50, 300], drops=[0.4, 0.1]).to(device)
#         self.classifier = self.classifier.to(device)
        
#         self.encoder = self.encoder.to(self.device)

        self.encoder = self.encoder.to(self.device)
        self.encoder.reset()

#         self.encoder_path.reset()
        
    
        # self.linear = torch.nn.Linear(1150,256).to(self.device)

    def train(self, data, optimizer, loss_fn, device):
        if self.pointwise:
            return self._train_pointwise_(data, optimizer, loss_fn, device)
        else:
            return self._train_pairwise_(data, optimizer, loss_fn, device)

    def _train_pointwise_(self, data, optimizer, loss_fn, device):
        self.encoder.train()

        ques_batch, path_batch, y_label = data['ques_batch'], data['path_batch'], data['y_label']

        ques_batch = tu.trim(ques_batch)
        path_batch = tu.trim(path_batch)

        h = self.encoder.init_hidden(ques_batch.shape[0])
        op_q = self.encoder(ques_batch.transpose(1, 0), h, return_h=True)
        op_p = self.encoder(path_batch.transpose(1, 0), h, return_h=True)

        ques_batch_encoded = op_q[-1]
        path_batch_encoded = op_p[-1]

        # Calculating dot score
        scores = torch.sum(ques_batch_encoded * path_batch_encoded, -1)

        loss = loss_fn(scores, y_label)
        torch.nn.utils.clip_grad_norm_(self.encoder.parameters(), .5)
        loss.backward()
        optimizer.step()
        return loss
        
    def _train_pairwise_(self, data, optimizer, loss_fn, device):
        '''
            Given data, passes it through model, inited in constructor, returns loss and updates the weight
            :params data: {batch of question, pos paths, neg paths and dummy y labels}
            :params optimizer: torch.optim object
            :params loss fn: torch.nn loss object
            :params device: torch.device object

            returns loss
        '''
        self.encoder.train()
        # Unpacking the data and model from args
        ques_batch, pos_batch, neg_batch, y_label = data['ques_batch'], data['pos_batch'], data['neg_batch'], data['y_label']
        
        optimizer.zero_grad()
        #Encoding all the data

#         print(f"Before: \t {ques_batch.shape[1]}, {pos_batch.shape[1]}, {neg_batch.shape[1]}\n"
#         f"After: \t\t {tu.trim(ques_batch).shape[1]}, {tu.trim(pos_batch).shape[1]}, {tu.trim(neg_batch).shape[1]}")
        
        ques_batch = tu.trim(ques_batch)
        pos_batch = tu.trim(pos_batch)
        neg_batch = tu.trim(neg_batch)
        
#         print(pos_batch)
#         print(neg_batch)

        h = self.encoder.init_hidden(ques_batch.shape[0])
        op_q = self.encoder(ques_batch.transpose(1,0), h, return_h=True)
#         h = self.encoder_path.init_hidden(ques_batch.shape[0],self.device)
        op_p = self.encoder(pos_batch.transpose(1,0),h, return_h=True)
        op_n = self.encoder(neg_batch.transpose(1,0),h, return_h=True)
                                   
        
#         self.encoder.reset_hidden()
#         op_q = self.encoder(ques_batch.transpose(1,0))[-1][-1][-1]
#         self.encoder.reset_hidden()
#         op_p = self.encoder(pos_batch.transpose(1,0))[-1][-1][-1]
#         self.encoder.reset_hidden()
#         op_n = self.encoder(neg_batch.transpose(1,0))[-1][-1][-1]
#         self.encoder.reset_hidden()
#         op_q = self.encoder(ques_batch.transpose(1,0))[1][-1][0].squeeze()
#         self.encoder.reset_hidden()
#         op_p = self.encoder(pos_batch.transpose(1,0))[1][-1][0].squeeze()
#         self.encoder.reset_hidden()
#         op_n = self.encoder(neg_batch.transpose(1,0))[1][-1][0].squeeze()

#         print(op_q.shape, op_p.shape, op_n.shape)
#         raise IOError
            
        ques_batch_encoded =  op_q[-1] 
        pos_batch_encoded =  op_p[-1]
        neg_batch_encoded =  op_n[-1]
        
#         ques_batch_encoded =  op_q[-1][0][1].squeeze()
#         pos_batch_encoded =  op_p[-1][0][1].squeeze()
#         neg_batch_encoded =  op_n[-1][0][1].squeeze()
        
#         print(ques_batch_encoded.shape, pos_batch_encoded.shape, neg_batch_encoded.shape)
#         raise IOError
        
        #Calculating dot score
        pos_scores = torch.sum(ques_batch_encoded * pos_batch_encoded, -1)
        neg_scores = torch.sum(ques_batch_encoded * neg_batch_encoded, -1)
#         print(pos_scores, neg_scores)
        
#         raise IOError
        
#         if True:
#             print("ques_batch shape is ", ques_batch.shape)
#             print("pos_batch shape is ", pos_batch.shape)
#             print("neg_batch shape is ", neg_batch.shape)
#             print("transposed ques bathc is ", ques_batch.transpose(1,0).shape)
#             for o in op_p[1]:
#                 print("o shape is ", o.shape)
#             print("encoded pos batch shape is ", op_p[1][-1][-1].shape)
            
#             print("pos_score is", pos_scores.shape)
        '''
            If `y == 1` then it assumed the first input should be ranked higher
            (have a larger value) than the second input, and vice-versa for `y == -1`
        '''
#         raise ValueError
        try:
            loss = loss_fn(pos_scores, neg_scores, y_label)
        except RuntimeError:
            traceback.print_exc()
            print(pos_scores.shape, neg_scores.shape, y_label.shape,  ques_batch.shape, pos_batch.shape, neg_batch.shape)

        loss.backward()
#         print([x.grad for x in modeler.encoder.parameters()])
#         print([x.grad for x in modeler.encoder_path.parameters()])
#         raise IOError
        torch.nn.utils.clip_grad_norm_(self.encoder.parameters(), .5)
        optimizer.step()
        return loss

    def predict(self, ques, paths, device):
        """
            Same code works for both pairwise or pointwise
        """
        with torch.no_grad():

#             print(ques.shape, paths.shape, tu.trim(ques).shape,tu.trim(paths).shape)
            
#             if tu.trim(ques).shape[0] == 0:
#                 print(ques)
#                 print(paths)
            
            self.encoder.eval()
#             self.encoder_path.eval()
#             h = self.encoder.reset()
            # Encoding all the data
            ques = tu.trim(ques)
            paths = tu.trim(paths)
    
            
            h  = self.encoder.init_hidden(ques.shape[0])
            op_q = self.encoder(ques.transpose(1,0), h, return_h=True,also_raw=True)
            
#             h = self.encoder_path.init_hidden(ques.shape[0],self.device)
            op_p = self.encoder(paths.transpose(1,0), h, return_h=True,also_raw=True)
#             self.encoder.reset_hidden()
#             op_q = self.encoder(ques.transpose(1,0))
#             self.encoder.reset_hidden()
#             op_p = self.encoder(paths.transpose(1,0))

#             question = op_q[1][0][-1]
#             paths = op_p[1][0][-1]
            question = op_q[-2]
            paths = op_p[-2]
        
#             question = op_q[-1][-1][-1]
#             paths = op_p[-1][-1][-1]
        
#             question = op_q[-1][0][1].squeeze()
#             paths = op_p[-1][0][1].squeeze()

            if self.pointwise:
                # question = F.normalize(F.relu(question),p=1,dim=1)
                # paths = F.normalize(F.relu(paths),p=1,dim=1)
                # norm_ques_batch = torch.abs(torch.norm(question, dim=1, p=1))
                # norm_pos_batch = torch.abs(torch.norm(paths, dim=1, p=1))
                score = torch.sum(question * paths, -1)
                # score = score.div(norm_ques_batch * norm_pos_batch).div_(2.0).add_(0.5)
            else:
                score = torch.sum(question * paths, -1)

            self.encoder.train()
#             self.encoder_path.train()
            return score
    
    def prepare_save(self):
        """

            This function is called when someone wants to save the underlying models.
            Returns a tuple of key:model pairs which is to be interpreted within save model.

        :return: [(key, model)]
        """
        return [('encoder', self.encoder)]

    def load_from(self, location):
        # Pull the data from disk
        if self.debug: print("loading Bilstmdot model from", location)
        self.encoder.load_state_dict(torch.load(location)['encoder'])
        if self.debug: print("model loaded with weights ,", self.get_parameter_sum())
    
    @property
    def layers(self):
        encoder_layer = [ l for l in self.encoder.layers]
#         encoder_path = [l for l in self.encoder_path.layers]
#         return torch.nn.ModuleList(encoder_layer+ encoder_path)
        return torch.nn.ModuleList(encoder_layer)

In [15]:
class QelosSlotPtrQuestionEncoder(nn.Module):
    def __init__(self, max_length, hidden_dim, number_of_layer,
                 embedding_dim, vocab_size, bidirectional, device,
                 dropout=0.0, mode='LSTM', enable_layer_norm=False,
                 vectors=None, residual=True, dropout_in=0., dropout_rec=0, debug=False):

        super(QelosSlotPtrQuestionEncoder, self).__init__()
        self.max_length, self.hidden_dim, self.embedding_dim, self.vocab_size = \
            int(max_length), int(hidden_dim), int(embedding_dim), int(vocab_size)
        self.enable_layer_norm = enable_layer_norm
        self.number_of_layer = number_of_layer
        self.bidirectional = bidirectional
        self.dropout = dropout
        self.dropout_in, self.dropout_rec = dropout_in, dropout_rec
        self.debug = debug
        self.mode = mode
        self.residual = residual
        self.device = device

        self.lstm = merity.RNNModel(rnn_type='LSTM', 
                ntoken=vocab_size,
                ninp=400,
                nhid=1150,
                nlayers=3,
                dropout=0.05,
                dropouth=0.2,
                dropouti=0.2,
                dropoute=0.05,
                wdrop=0,
                tie_weights=True)
        self.lstm.encoder.padding_idx = 0 
        
        key_mapping = {
                    'encoder.weight' : 'encoder.weight',
                    'rnns.0.module.weight_ih_l0' : 'rnns.0.weight_ih_l0', 
                    'rnns.0.module.bias_ih_l0' : 'rnns.0.bias_ih_l0', 
                    'rnns.0.module.bias_hh_l0' : 'rnns.0.bias_hh_l0', 
                    'rnns.0.module.weight_hh_l0_raw' : 'rnns.0.weight_hh_l0',
                    'rnns.1.module.weight_ih_l0' : 'rnns.1.weight_ih_l0', 
                    'rnns.1.module.bias_ih_l0' : 'rnns.1.bias_ih_l0', 
                    'rnns.1.module.bias_hh_l0': 'rnns.1.bias_hh_l0', 
                    'rnns.1.module.weight_hh_l0_raw' : 'rnns.1.weight_hh_l0', 
                    'rnns.2.module.weight_ih_l0' : 'rnns.2.weight_ih_l0', 
                    'rnns.2.module.bias_ih_l0' : 'rnns.2.bias_ih_l0', 
                    'rnns.2.module.bias_hh_l0': 'rnns.2.bias_hh_l0', 
                    'rnns.2.module.weight_hh_l0_raw' : 'rnns.2.weight_hh_l0',
                }
        
        # Load the pre-trained model
        # pretrained_weights = torch.load('./resources/unsup_model_enc.torch', map_location= lambda storage, loc: storage)
        # pretrained_weights = torch.load('./resources/pretr_model_enc.torch', map_location= lambda storage, loc: storage)
        pretrained_weights = torch.load('./ulmfit/wt103/fwd_wt103_enc.h5', map_location= lambda storage, loc: storage)
        # pretrained_weights = torch.load(LM_DIR /'unsup_model_enc.torch', map_location= lambda storage, loc: storage)

        new_vectors = vectors
        pretrained_weights['encoder.weight'] = T(new_vectors)
        pretrained_weights.pop('encoder_with_dropout.embed.weight')
        
        for k,v in key_mapping.items():
            pretrained_weights[v] = pretrained_weights.pop(k)

        self.lstm.load_state_dict(pretrained_weights)

        dims = [self.hidden_dim]
        self.linear = torch.nn.Linear(dims[-1] * (1+self.bidirectional), 2)
        self.sm = torch.nn.Softmax(1)
        # for adapter
        outdim = dims[-1] * (1+self.bidirectional)
        self.adapt_lin = None

        if outdim != self.embedding_dim:
            self.adapt_lin = torch.nn.Linear(self.embedding_dim, outdim, bias=False)

    @property
    def layers(self):
        layers = [x for x in self.lstm.layers]
        layers.append(torch.nn.ModuleList([self.linear]))
        return torch.nn.ModuleList(layers)
    
    def return_encoder(self):
        return self.lstm

    def forward(self, x, predict = False):
        
        x = tu.trim(x)
        
        h = self.lstm.init_hidden(x.shape[0])
        if predict:
            _, ys, _, embs, _, _ = self.lstm(x.transpose(1, 0), h, return_h=True, also_raw=True)
        else:
            ys, _, embs, _ = self.lstm(x.transpose(1, 0), h, return_h=True)
        mask = tu.compute_mask(x)

        ys = ys[-1].transpose(1,0)

        # Get attention scores
        linear_scores = self.linear(ys)

        scorer = linear_scores + torch.log(mask[:, :ys.size(1)].float().unsqueeze(2))
        scores = self.sm(scorer)  # (batsize, seqlen, 2)

        # get summaries
        # region skipper
        skipadd = embs[:, :ys.size(1), :]
        if self.adapt_lin is not None:
            skipadd = self.adapt_lin(skipadd)
        if not self.residual:
            ys = ys + skipadd
        # endregion

        ys = ys.unsqueeze(2)  # (batsize, seqlen, 1, dim)
        scores = scores.unsqueeze(3)  # (batsize, seqlen, 2, 1)
        b = ys * scores  # (batsize, seqlen, 2, dim)
        summaries = b.sum(1)  # (batsize, 2, dim)
        ret = torch.cat([summaries[:, 0, :], summaries[:, 1, :]], 1)
        return ret,scores

In [16]:
class QelosFlatEncoder(nn.Module):
    def __init__(self, max_length, hidden_dim, number_of_layer,
                 embedding_dim, vocab_size, bidirectional, device,
                 dropout=0.0, mode='LSTM', enable_layer_norm=False,
                 vectors=None, residual=False, dropout_in=0., dropout_rec=0, debug=False,encoder=False):
        '''
               :param max_length: Max length of the sequence.
               :param hidden_dim: dimension of the output of the LSTM.
               :param number_of_layer: Number of LSTM to be stacked.
               :param embedding_dim: The output dimension of the embedding layer/ important only if vectors=none
               :param vocab_size: Size of vocab / number of rows in embedding matrix
               :param bidirectional: boolean - if true creates BIdir LStm
               :param vectors: embedding matrix
               :param debug: Bool/ prints shapes and some other meta data.
               :param enable_layer_norm: Bool/ layer normalization.
               :param mode: LSTM/GRU.
               :param residual: Bool/ return embedded state of the input.

           TODO: Implement multilayered shit someday.
        '''
        super(QelosFlatEncoder, self).__init__()

        self.max_length, self.hidden_dim, self.embedding_dim, self.vocab_size = \
            int(max_length), int(hidden_dim), int(embedding_dim), int(vocab_size)
        self.enable_layer_norm = enable_layer_norm
        self.number_of_layer = number_of_layer
        self.bidirectional = bidirectional
        self.dropout = dropout
        self.dropout_in, self.dropout_rec = dropout_in, dropout_rec
        self.debug = debug
        self.mode = mode
        self.residual = residual
        self.device = device
        
        
        if not encoder:
            self.lstm = merity.RNNModel(rnn_type='LSTM', 
                    ntoken=self.vocab_size,
                    ninp=400,
                    nhid=1150,
                    nlayers=3,
                    dropout=0.1,
                    dropouth=0.4,
                    dropouti=0.4,
                    dropoute=0.1,
                    wdrop=0,
                    tie_weights=True)
            self.lstm.encoder.padding_idx = 0 
        else:
            self.lstm = encoder
        key_mapping = {
                    'encoder.weight' : 'encoder.weight',
                    'rnns.0.module.weight_ih_l0' : 'rnns.0.weight_ih_l0', 
                    'rnns.0.module.bias_ih_l0' : 'rnns.0.bias_ih_l0', 
                    'rnns.0.module.bias_hh_l0' : 'rnns.0.bias_hh_l0', 
                    'rnns.0.module.weight_hh_l0_raw' : 'rnns.0.weight_hh_l0',
                    'rnns.1.module.weight_ih_l0' : 'rnns.1.weight_ih_l0', 
                    'rnns.1.module.bias_ih_l0' : 'rnns.1.bias_ih_l0', 
                    'rnns.1.module.bias_hh_l0': 'rnns.1.bias_hh_l0', 
                    'rnns.1.module.weight_hh_l0_raw' : 'rnns.1.weight_hh_l0', 
                    'rnns.2.module.weight_ih_l0' : 'rnns.2.weight_ih_l0', 
                    'rnns.2.module.bias_ih_l0' : 'rnns.2.bias_ih_l0', 
                    'rnns.2.module.bias_hh_l0': 'rnns.2.bias_hh_l0', 
                    'rnns.2.module.weight_hh_l0_raw' : 'rnns.2.weight_hh_l0',
                }
        
        # Load the pre-trained model
        # pretrained_weights = torch.load('./resources/unsup_model_enc.torch', map_location= lambda storage, loc: storage)
        # pretrained_weights = torch.load('./resources/pretr_model_enc.torch', map_location= lambda storage, loc: storage)
        pretrained_weights = torch.load('./ulmfit/wt103/fwd_wt103_enc.h5', map_location= lambda storage, loc: storage)
        # pretrained_weights = torch.load(LM_DIR /'unsup_model_enc.torch', map_location= lambda storage, loc: storage)

        new_vectors = vectors
        pretrained_weights['encoder.weight'] = T(new_vectors)
        pretrained_weights.pop('encoder_with_dropout.embed.weight')

        self.adapt_lin = None   # Make layer if dims mismatch
        if residual and self.hidden_dim*2 != self.embedding_dim:
            self.adapt_lin = torch.nn.Linear(self.embedding_dim, self.hidden_dim*2, bias=False)
    
    @property
    def layers(self):
        return self.lstm.layers
            
    def forward(self, x, predict=False):
        # embs = self.embedding_layer(x)
        # mask = tu.compute_mask(x)

#         h = self.lstm.init_hidden(x.shape[0],self.device)

#         if self.residual:
#             _, final_state, _, mask, embs, _ = self.lstm(x, h)
#         else:
#             _, final_state, _, mask = self.lstm(x, h)
        
        h = self.lstm.init_hidden(x.shape[0])
        op = self.lstm(x.transpose(1,0), h, return_h=True, also_raw=predict)
        
        if predict:
            final_state = op[5]
        else:
            final_state = op[3]
#         print(f"final state shape is, {final_state.shape}")
        
#         print([(x[0].shape, x[1].shape) for x in final_state])
        # final_state = self.lstm.y_n[-1]
        final_state = final_state.contiguous().view(x.size(0), -1)

        # if self.residual:
        #     if self.adapt_lin is not None:
        #         embs = self.adapt_lin(embs)
        #     meanpool = embs.sum(0)
        #     masksum = mask.float().sum(1).unsqueeze(1)
        #     meanpool = meanpool / masksum
        #     final_state = final_state + meanpool
        return final_state

class QelosSlotPtrChainEncoder(nn.Module):
    def __init__(self, max_length, hidden_dim, number_of_layer,
                 embedding_dim, vocab_size, bidirectional, device,
                 dropout=0.0, mode='LSTM', enable_layer_norm=False,
                 vectors=None, residual=False, dropout_in=0., dropout_rec=0,debug=False,encoder=False):
        '''
               :param max_length: Max length of the sequence.
               :param hidden_dim: dimension of the output of the LSTM.
               :param number_of_layer: Number of LSTM to be stacked.
               :param embedding_dim: The output dimension of the embedding layer/ important only if vectors=none
               :param vocab_size: Size of vocab / number of rows in embedding matrix
               :param bidirectional: boolean - if true creates BIdir LStm
               :param vectors: embedding matrix
               :param debug: Bool/ prints shapes and some other meta data.
               :param enable_layer_norm: Bool/ layer normalization.
               :param mode: LSTM/GRU.
               :param residual: Bool/ return embedded state of the input.

           TODO: Implement multilayered shit someday.
        '''
        super(QelosSlotPtrChainEncoder, self).__init__()

        self.max_length, self.hidden_dim, self.embedding_dim, self.vocab_size = \
            int(max_length), int(hidden_dim), int(embedding_dim), int(vocab_size)
        self.enable_layer_norm = enable_layer_norm
        self.number_of_layer = number_of_layer
        self.bidirectional = bidirectional
        self.dropout = dropout
        self.dropout_in, self.dropout_rec = dropout_in, dropout_rec
        self.debug = debug
        self.mode = mode
        self.residual = residual
        self.device = device
        
        self.enc = QelosFlatEncoder(max_length, hidden_dim, number_of_layer,
                 embedding_dim, vocab_size, bidirectional, device, dropout=0.5, mode='LSTM',
                 enable_layer_norm=False, vectors=vectors, residual=self.residual,
                 dropout_in=self.dropout_in, dropout_rec=self.dropout_rec, debug=False,encoder=encoder)#.to(device)

    def forward(self, firstrels, secondrels, predict=False):
#         if True:
#             print(firstrels.shape,secondrels.shape)
#             print(f"first and secod respectively, {firstrels.shape},{secondrels.shape}")
        firstrels_enc = self.enc(firstrels, predict)
        secondrels_enc = self.enc(secondrels, predict)
#         print(f"first and secod respectively, {firstrels_enc.shape},{secondrels_enc.shape}")
        # cat???? # TODO
        enc = torch.cat([firstrels_enc, secondrels_enc], 1)
        return enc
    
    @property
    def layers(self):
        return self.enc.layers

In [17]:
class QelosSlotPointerModel(net.Model):

    """
        Eating Denis's shit
    """
    def __init__(self, _parameter_dict, _word_to_id, _device, _pointwise=False, _debug=False):

        self.debug = _debug
        self.parameter_dict = _parameter_dict
        self.device = _device
        self.pointwise = _pointwise
        self.word_to_id = _word_to_id

        if self.debug:
            print("Init Models")

        self.encoder_q = QelosSlotPtrQuestionEncoder(
            number_of_layer=self.parameter_dict['number_of_layer'],
            bidirectional=self.parameter_dict['bidirectional'],
            embedding_dim=self.parameter_dict['embedding_dim'],
            hidden_dim=self.parameter_dict['hidden_size'],
            vocab_size=self.parameter_dict['vocab_size'],
            max_length=self.parameter_dict['max_length'],
            dropout=self.parameter_dict['dropout'],
            vectors=self.parameter_dict['vectors'],
            enable_layer_norm=False,
            device=_device,
            residual=True,
            mode = 'LSTM',
            dropout_in=self.parameter_dict['dropout_in'],
            dropout_rec=self.parameter_dict['dropout_rec'],
            debug = self.debug).to(self.device)

        enc = self.encoder_q.return_encoder()
        
        self.encoder_p = QelosSlotPtrChainEncoder(
            number_of_layer=self.parameter_dict['number_of_layer'],
            embedding_dim=self.parameter_dict['embedding_dim'],
            bidirectional=self.parameter_dict['bidirectional'],
            hidden_dim=self.parameter_dict['hidden_size'],
            vocab_size=self.parameter_dict['vocab_size'],
            max_length=self.parameter_dict['max_length'],
            dropout=self.parameter_dict['dropout'],
            vectors=self.parameter_dict['vectors'],
            enable_layer_norm=False,
            device=_device,
            residual=False,
            mode = 'LSTM',
            dropout_in=self.parameter_dict['dropout_in'],
            dropout_rec=self.parameter_dict['dropout_rec'],
            debug = self.debug,
            encoder = enc).to(self.device)
    
    @property
    def layers(self):
        return torch.nn.ModuleList(
#             [l for l in self.encoder_p.layers] +
            [l for l in self.encoder_q.layers]
        )

    def train(self, data, optimizer, loss_fn, device, predict=False):
        if self.pointwise:
            return self._train_pointwise_(data, optimizer, loss_fn, device, predict)
        else:
            return self._train_pairwise_(data, optimizer, loss_fn, device, predict)

    def _train_pairwise_(self, data, optimizer, loss_fn, device, predict):
        ques_batch, pos_1_batch, pos_2_batch, neg_1_batch, neg_2_batch, y_label = \
            data['ques_batch'], data['pos_rel1_batch'], data['pos_rel2_batch'], \
            data['neg_rel1_batch'], data['neg_rel2_batch'], data['y_label']

        optimizer.zero_grad()

        # Have to manually check if the 2nd paths holds anything in this batch.
        # If not, we have to pad everything up with zeros, or even call a limited part of the comparison module.
        pos_2_batch = tu.no_one_left_behind(pos_2_batch)
        neg_2_batch = tu.no_one_left_behind(neg_2_batch)

        # Encoding all the data
#         if True:
#             print(f"Shape of ques trimmed is: {tu.trim(ques_batch).shape}")
#             print(f"Shape of pos1 trimmed is: {tu.trim(pos_1_batch).shape}")
#             print(f"Shape of pos2 trimmed is: {tu.trim(pos_2_batch).shape}")
        ques_encoded,_ = self.encoder_q(tu.trim(ques_batch), predict)
        pos_encoded = self.encoder_p(tu.trim(pos_1_batch), tu.trim(pos_2_batch), predict)
        neg_encoded = self.encoder_p(tu.trim(neg_1_batch), tu.trim(neg_2_batch), predict)

        # Pass them to the comparison module
        pos_scores = torch.sum(ques_encoded * pos_encoded, dim=-1)
        neg_scores = torch.sum(ques_encoded * neg_encoded, dim=-1)

        '''
            If `y == 1` then it assumed the first input should be ranked higher
            (have a larger value) than the second input, and vice-versa for `y == -1`
        '''
        loss = loss_fn(pos_scores, neg_scores, y_label)
        loss.backward()
        optimizer.step()

        return loss

    def _train_pointwise_(self, data, optimizer, loss_fn, device, predict):
        ques_batch, path_1_batch, path_2_batch, y_label = \
            data['ques_batch'], data['path_rel1_batch'], data['path_rel2_batch'], data['y_label']


        optimizer.zero_grad()
        path_2_batch = tu.no_one_left_behind(path_2_batch)

        # Encoding all the data
        ques_encoded,_ = self.encoder_q(tu.trim(ques_batch), predict)
        pos_encoded = self.encoder_p(tu.trim(path_1_batch), tu.trim(path_2_batch), predict)

        score = torch.sum(ques_encoded * pos_encoded, dim=-1)

        '''
            Binary Cross Entropy loss function. 
        '''
        loss = loss_fn(score, y_label)
        loss.backward()
        optimizer.step()

        return loss

    def predict(self, ques, paths, paths_rel1,paths_rel2, device,attention_value=False):
        """
            Same code works for both pairwise or pointwise
        """
        with torch.no_grad():
            self.encoder_q.eval()
            self.encoder_p.eval()

            # Have to manually check if the 2nd paths holds anything in this batch.
            # If not, we have to pad everything up with zeros, or even call a limited part of the comparison module.
            paths_rel2 = tu.no_one_left_behind(paths_rel2)

            # Encoding all the data
            ques_encoded,attention_score = self.encoder_q(tu.trim(ques), predict=True)
            path_encoded = self.encoder_p(tu.trim(paths_rel1), tu.trim(paths_rel2), predict=True)

            # Pass them to the comparison module
            score = torch.sum(ques_encoded * path_encoded, dim=-1)

            self.encoder_q.train()
            self.encoder_p.train()
            if attention_value:
                return score,attention_score
            else:
                return score

    def prepare_save(self):
        return [('encoder_q', self.encoder_q), ('encoder_p', self.encoder_p)]

In [18]:
parameter_dict['vectors'] = data['vectors']
parameter_dict['vocab_size'] = data['vectors'].shape[0]
parameter_dict['schema'] = schema
parameter_dict['bidirectional'] = False
parameter_dict['hidden_size'] = data['vectors'].shape[1]

In [19]:
itos = pickle.load(open('./ulmfit/wt103/itos_wt103.pkl', 'rb'))
# itos = pickle.load(open('./resources/itos.pkl', 'rb'))
# itos = pickle.load(open(LM_DIR/'itos.pkl', 'rb'))


# new_w = torch.load('./resources/unsup_model_enc.torch')['encoder.weight'].cpu().data.numpy()
# new_w = torch.load('./resources/pretr_model_enc.torch')['encoder.weight'].cpu().data.numpy()
new_w = torch.load('./ulmfit/wt103/fwd_wt103_enc.h5', map_location= lambda storage, loc: storage)['encoder.weight'].cpu().data.numpy()
# new_w = torch.load(LM_DIR/'unsup_model_enc.torch', map_location= lambda storage, loc: storage)['encoder.weight'].cpu().data.numpy()
old_w = parameter_dict['vectors']
print(f"{len(itos)}, {new_w.shape}, {old_w.shape}")
for i, w in enumerate(itos):
    
    if w not in ei.vocab:
        continue

    old_w[ei.vocab[w]] = new_w[i]
    
parameter_dict['vectors'] = old_w

238462, (238462, 400), (238462, 400)


In [20]:
modeler = QelosSlotPointerModel(_parameter_dict=parameter_dict, _word_to_id=_word_to_id,
                                                _device=device, _pointwise=pointwise, _debug=False)
# optimizer = optim.Adam(list(filter(lambda p: p.requires_grad, modeler.encoder_q.parameters())) +
#                        list(filter(lambda p: p.requires_grad, modeler.encoder_p.parameters())), weight_decay=0.0001, lr=0.01)

[LSTM(400, 1150), LSTM(1150, 1150), LSTM(1150, 400)]


In [21]:
# best_model_path = Path('data/models/core_chain/slotptr/lcquad/173/model.torch')
# model_weights = torch.load(best_model_path)
# modeler.encoder_p.load_state_dict(model_weights['encoder_p'])
# modeler.encoder_q.load_state_dict(model_weights['encoder_q'])

In [22]:
# modeler = BiLstmDot_ulmfit(_parameter_dict = parameter_dict,
#                     _word_to_id=_word_to_id,
#                     _device=device,
#                     _pointwise=pointwise,
#                     _debug=False)
opt_fn = partial(optim.Adam, betas=(0.7, 0.99))
optimizer = make_opt(modeler, opt_fn, lr=0.00)
optimizer.param_groups[-1]['lr'] = 0.01
# optimizer.param_groups[-2]['lr'] = 0.001
# optimizer = optim.Adam(
#     list(filter(lambda p: p.requires_grad, modeler.encoder.parameters()))+
#                       list(filter(lambda p: p.requires_grad, modeler.encoder_path.parameters())))

In [23]:
optimizer.param_groups

[{'params': [],
  'lr': 0.0,
  'betas': (0.7, 0.99),
  'eps': 1e-08,
  'weight_decay': 0,
  'amsgrad': False},
 {'params': [Parameter containing:
   tensor([[ 1.4854e-05, -2.3424e-05,  1.9693e-05,  ...,  2.1349e-05,
             2.1776e-05, -1.2394e-05],
           [-1.2274e-01,  2.7886e-01, -3.8850e-01,  ..., -1.0404e-01,
             1.9580e-02,  1.8548e-01],
           [ 1.8070e-01,  1.5874e+00, -1.1738e-01,  ..., -4.5935e-02,
            -8.1352e-02,  1.8054e-01],
           ...,
           [-1.8595e-03, -6.8529e-03,  1.6999e-03,  ...,  1.7039e-03,
             4.1632e-03, -1.3171e-03],
           [-2.3120e-03, -6.9001e-03,  1.8772e-03,  ...,  5.0309e-04,
             4.6596e-03, -2.5850e-03],
           [-2.2463e-03, -9.1512e-03,  1.3927e-03,  ...,  1.2296e-03,
             5.8085e-03, -1.8940e-03]], device='cuda:0', requires_grad=True)],
  'lr': 0.0,
  'betas': (0.7, 0.99),
  'eps': 1e-08,
  'weight_decay': 0,
  'amsgrad': False},
 {'params': [Parameter containing:
   tensor([[-0

In [24]:
for iterations, sample_batched in enumerate(train_loader):
    pass

In [25]:
def validation_accuracy(valid_questions, valid_pos_paths, valid_neg_paths,  modeler, device,return_traces, *path_rel):

    if path_rel:
        valid_pos_paths_rel1, valid_pos_paths_rel2, valid_neg_paths_rel1, valid_neg_paths_rel2 = path_rel[0],path_rel[1],path_rel[2],path_rel[3]
    
    correct, incorrect = [], []
    precision = []
    with torch.no_grad():
        for i in range(len(valid_questions)):
            question = np.repeat(valid_questions[i].reshape(1, -1), len(valid_neg_paths[i]) + 1,
                                 axis=0)  # +1 for positive path
            # paths = np.vstack((valid_pos_paths[i].reshape(1, -1), valid_neg_paths[i]))
            paths = np.vstack((valid_neg_paths[i],valid_pos_paths[i].reshape(1, -1)))
            if path_rel:
                paths_rel1 = np.vstack((valid_neg_paths_rel1[i],valid_pos_paths_rel1[i].reshape(1, -1)))
                paths_rel2 = np.vstack((valid_neg_paths_rel2[i],valid_pos_paths_rel2[i].reshape(1, -1)))
                paths_rel1 = torch.tensor(paths_rel1, dtype=torch.long, device=device)
                paths_rel2 = torch.tensor(paths_rel2, dtype=torch.long, device=device)

            question = torch.tensor(question, dtype=torch.long, device=device)
            paths = torch.tensor(paths, dtype=torch.long, device=device)

            if path_rel:
                score = modeler.predict(question, paths, paths_rel1, paths_rel2, device)
            else:
                score = modeler.predict(question, paths, device)
            arg_max = torch.argmax(score)
            if arg_max.item() == len(paths)-1:  # 0 is the positive path index
                precision.append(1)
                correct.append([question,paths])
            else:
                precision.append(0)
                incorrect.append([question,paths])
    if return_traces:
        return correct,incorrect,sum(precision) * 1.0 / len(precision)
    return sum(precision) * 1.0 / len(precision)

In [26]:
def curatail_padding(data,parameter_dict):

    '''
        Since schema is already implicitly defined/present in the parameter_dict['rel1_pad']
    '''

    data['test_neg_paths'] = data['test_neg_paths'][:, :, :parameter_dict['rel_pad']]
    data['test_pos_paths'] = data['test_pos_paths'][:, :parameter_dict['rel_pad']]

    if parameter_dict['schema'] == 'reldet':
        data['test_neg_paths_rel1_rd'] = data['test_neg_paths_rel1_rd'][:, :, :parameter_dict['rel1_pad']]
        data['test_neg_paths_rel2_rd'] = data['test_neg_paths_rel2_rd'][:, :, :parameter_dict['rel1_pad']]
        data['test_pos_paths_rel1_rd'] = data['test_pos_paths_rel1_rd'][:, :parameter_dict['rel1_pad']]
        data['test_pos_paths_rel2_rd'] = data['test_pos_paths_rel2_rd'][:, :parameter_dict['rel1_pad']]
    elif parameter_dict['schema'] == 'slotptr':
        data['test_neg_paths_rel1_sp'] = data['test_neg_paths_rel1_sp'][:, :, :parameter_dict['rel1_pad']]
        data['test_neg_paths_rel2_sp'] = data['test_neg_paths_rel2_sp'][:, :, :parameter_dict['rel1_pad']]
        data['test_pos_paths_rel1_sp'] = data['test_pos_paths_rel1_sp'][:, :parameter_dict['rel1_pad']]
        data['test_pos_paths_rel2_sp'] = data['test_pos_paths_rel2_sp'][:, :parameter_dict['rel1_pad']]

    data['valid_neg_paths'] = data['valid_neg_paths'][:, :, :parameter_dict['rel_pad']]
    data['valid_pos_paths'] = data['valid_pos_paths'][:, :parameter_dict['rel_pad']]

    if parameter_dict['schema'] == 'reldet':
        data['valid_neg_paths_rel1_rd'] = data['valid_neg_paths_rel1_rd'][:, :, :parameter_dict['rel1_pad']]
        data['valid_neg_paths_rel2_rd'] = data['valid_neg_paths_rel2_rd'][:, :, :parameter_dict['rel1_pad']]
        data['valid_pos_paths_rel1_rd'] = data['valid_pos_paths_rel1_rd'][:, :parameter_dict['rel1_pad']]
        data['valid_pos_paths_rel2_rd'] = data['valid_pos_paths_rel2_rd'][:, :parameter_dict['rel1_pad']]
    elif parameter_dict['schema'] == 'slotptr':
        data['valid_neg_paths_rel1_sp'] = data['valid_neg_paths_rel1_sp'][:, :, :parameter_dict['rel1_pad']]
        data['valid_neg_paths_rel2_sp'] = data['valid_neg_paths_rel2_sp'][:, :, :parameter_dict['rel1_pad']]
        data['valid_pos_paths_rel1_sp'] = data['valid_pos_paths_rel1_sp'][:, :parameter_dict['rel1_pad']]
        data['valid_pos_paths_rel2_sp'] = data['valid_pos_paths_rel2_sp'][:, :parameter_dict['rel1_pad']]

    return data

In [27]:
def training_loop(training_model, parameter_dict,modeler,train_loader,
                  optimizer,loss_func, data, dataset, device, test_every, validate_every, lrschedule,
                  pointwise = False, problem='core_chain', curtail_padding_rel=True):

    model_save_location = aux.save_location(problem, training_model, dataset)
    aux_save_information = {
        'epoch' : 0,
        'test_accuracy':0.0,
        'validation_accura""cy':0.0,
        'parameter_dict':parameter_dict
    }
    train_loss = []
    valid_accuracy = []
    test_accuracy = []
    best_validation_accuracy = 0
    best_test_accuracy = 0

    if parameter_dict['schema'] == 'reldet':
        parameter_dict['rel1_pad'] =  parameter_dict['relrd_pad']
    elif parameter_dict['schema'] == 'slotptr':
        parameter_dict['rel1_pad'] = parameter_dict['relsp_pad']

        ###############
    # Training Loop
    ###############


    #Makes test data of appropriate shape
    print("the dataset is ", dataset)
    if curtail_padding_rel and dataset == 'lcquad':
        data = curatail_padding(data, parameter_dict)
#     data['valid_neg_paths'] = np.zeros_like(data['valid_neg_paths']) + 10
#     data['valid_pos_paths'] = np.zeros_like(data['valid_pos_paths']) + 10
#     data['valid_questions'] = np.zeros_like(data['valid_questions']) + 10

    try:

        for epoch in range(parameter_dict['epochs']):

            # Epoch start print
            print("Epoch: ", epoch, "/", parameter_dict['epochs'])

            # Bookkeeping variables
            epoch_loss = []
            epoch_time = time.time()

            # Loop for one batch
            # tqdm_loop = tqdm(enumerate(train_loader))
            for i_batch, sample_batched in enumerate(train_loader):

                # Bookkeeping and data preparation
                batch_time = time.time()
                update_lr(optimizer,lrschedule.get())


                if not pointwise:
                    ques_batch = torch.tensor(np.reshape(sample_batched[0][0], (-1, parameter_dict['max_length'])),
                                              dtype=torch.long, device=device)
                    pos_batch = torch.tensor(np.reshape(sample_batched[0][1], (-1, parameter_dict['max_length'])),
                                             dtype=torch.long, device=device)
                    neg_batch = torch.tensor(np.reshape(sample_batched[0][2], (-1, parameter_dict['max_length'])),
                                             dtype=torch.long, device=device)

                    data['dummy_y'] = torch.ones(ques_batch.shape[0], device=device)
                    if parameter_dict['schema'] != 'default':
                        pos_rel1_batch = torch.tensor(np.reshape(sample_batched[0][3], (-1, parameter_dict['max_length'])),
                                                      dtype=torch.long, device=device)
                        pos_rel2_batch = torch.tensor(np.reshape(sample_batched[0][4], (-1, parameter_dict['max_length'])),
                                                      dtype=torch.long, device=device)
                        neg_rel1_batch = torch.tensor(np.reshape(sample_batched[0][5], (-1, parameter_dict['max_length'])),
                                                      dtype=torch.long, device=device)
                        neg_rel2_batch = torch.tensor(np.reshape(sample_batched[0][6], (-1, parameter_dict['max_length'])),
                                                      dtype=torch.long, device=device)


                        data_batch = {
                            'ques_batch': ques_batch,
                            'pos_batch': pos_batch[:,:parameter_dict['rel_pad']],
                            'neg_batch': neg_batch[:,:parameter_dict['rel_pad']],
                            'y_label': data['dummy_y'],
                            'pos_rel1_batch': pos_rel1_batch[:,:parameter_dict['rel1_pad']],
                            'pos_rel2_batch':pos_rel2_batch[:,:parameter_dict['rel1_pad']],
                            'neg_rel1_batch':neg_rel1_batch[:,:parameter_dict['rel1_pad']],
                            'neg_rel2_batch' : neg_rel2_batch[:,:parameter_dict['rel1_pad']]
                        }

                    else:

                        data_batch = {
                            'ques_batch': ques_batch,
                            'pos_batch': pos_batch[:,:parameter_dict['rel_pad']],
                            'neg_batch': neg_batch[:,:parameter_dict['rel_pad']],
                            'y_label': data['dummy_y']}

                else:
                    ques_batch = torch.tensor(np.reshape(sample_batched[0][0], (-1, parameter_dict['max_length'])),
                                              dtype=torch.long, device=device)
                    path_batch = torch.tensor(np.reshape(sample_batched[0][1], (-1, parameter_dict['max_length'])),
                                             dtype=torch.long, device=device)
                    y = torch.tensor(sample_batched[1],dtype = torch.float,device=device).view(-1)


                    if parameter_dict['schema'] != 'default':
                        path_rel1_batch = torch.tensor(np.reshape(sample_batched[0][2], (-1, parameter_dict['max_length'])),
                                                      dtype=torch.long, device=device)
                        path_rel2_batch = torch.tensor(np.reshape(sample_batched[0][3], (-1, parameter_dict['max_length'])),
                                                      dtype=torch.long, device=device)

                        data_batch = {
                            'ques_batch': ques_batch,
                            'path_batch': path_batch[:,:parameter_dict['rel_pad']],
                            'y_label': y,
                            'path_rel1_batch': path_rel1_batch[:,:parameter_dict['rel1_pad']],
                            'path_rel2_batch': path_rel2_batch[:,:parameter_dict['rel1_pad']]
                        }
                    else:
                        data_batch = {
                            'ques_batch': ques_batch,
                            'path_batch': path_batch[:,:parameter_dict['rel_pad']],
                            'y_label': y
                        }



                loss = modeler.train(data=data_batch,
                                  optimizer=optimizer,
                                  loss_fn=loss_func,
                                  device=device)

                # Bookkeep the training loss
                epoch_loss.append(loss.item())

                # tqdm_loop.desc("#"+str(i_batch)+"\tLoss:" + str(loss.item())[:min(5, len(str(loss.item())))])

                print("Batch:\t%d" % i_batch, "/%d\t: " % (parameter_dict['batch_size']),
                      "%s" % (time.time() - batch_time),
                      "\t%s" % (time.time() - epoch_time),
                      "\t%s" % (str(loss.item())),
                      end=None if i_batch + 1 == int(int(i_batch) / parameter_dict['batch_size']) else "\n")

            # EPOCH LEVEL

            # Track training loss
            train_loss.append(sum(epoch_loss))

            # test_every = False
            if test_every:
                # Run on test set
                if epoch%test_every == 0:
                    if parameter_dict['schema'] != 'default':
                        if parameter_dict['schema']  == 'slotptr':
                            test_accuracy.append(aux.validation_accuracy(data['test_questions'], data['test_pos_paths'],
                                                             data['test_neg_paths'],modeler, device,data['test_pos_paths_rel1_sp'],data['test_pos_paths_rel2_sp'],
                                                                 data['test_neg_paths_rel1_sp'],data['test_neg_paths_rel2_sp']))
                        else:
                            test_accuracy.append(aux.validation_accuracy(data['test_questions'], data['test_pos_paths'],
                                                                         data['test_neg_paths'], modeler, device,
                                                                         data['test_pos_paths_rel1_rd'],
                                                                         data['test_pos_paths_rel2_rd'],
                                                                         data['test_neg_paths_rel1_rd'],
                                                                         data['test_neg_paths_rel2_rd']))
                    else:
                        test_accuracy.append(aux.validation_accuracy(data['test_questions'], data['test_pos_paths'],
                                                                     data['test_neg_paths'], modeler, device))
                    if test_accuracy[-1] >= best_test_accuracy:
                        best_test_accuracy = test_accuracy[-1]
                        aux_save_information['test_accuracy'] = best_test_accuracy
            else:
                test_accuracy.append(0)
                best_test_accuracy = 0

            # Run on validation set
            if validate_every:
                if epoch%validate_every == 0:
                    if parameter_dict['schema'] != 'default':
                        if parameter_dict['schema'] == 'slotptr':
                            valid_accuracy.append(aux.validation_accuracy(data['valid_questions'], data['valid_pos_paths'],
                                                              data['valid_neg_paths'],  modeler, device, data['valid_pos_paths_rel1_sp'],data['valid_pos_paths_rel2_sp'],
                                                                 data['valid_neg_paths_rel1_sp'],data['valid_neg_paths_rel2_sp']))
                        else:
                            valid_accuracy.append(aux.validation_accuracy(data['valid_questions'][:-1], data['valid_pos_paths'][:-1],
                                                                          data['valid_neg_paths'][:-1], modeler, device,
                                                                          data['valid_pos_paths_rel1_rd'][:-1],
                                                                          data['valid_pos_paths_rel2_rd'][:-1],
                                                                          data['valid_neg_paths_rel1_rd'][:-1],
                                                                          data['valid_neg_paths_rel2_rd'][:-1]))
                    else:
                        valid_accuracy.append(aux.validation_accuracy(data['valid_questions'], data['valid_pos_paths'],
                                                                      data['valid_neg_paths'], modeler, device))
                    if valid_accuracy[-1] > best_validation_accuracy:
                        print("MODEL WEIGHTS RIGHT NOW: ", modeler.get_parameter_sum())
                        best_validation_accuracy = valid_accuracy[-1]
                        aux_save_information['epoch'] = epoch
                        aux_save_information['validation_accuracy'] = best_validation_accuracy
                        aux.save_model(model_save_location, modeler, model_name='model.torch'
                                   , epochs=epoch, optimizer=optimizer, accuracy=best_validation_accuracy, aux_save_information=aux_save_information)

            # Resample new negative paths per epoch and shuffle all data
            train_loader.dataset.shuffle()

            # Epoch level prints
            print("Time: %s\t" % (time.time() - epoch_time),
                  "Loss: %s\t" % (sum(epoch_loss)),
                  "Valdacc: %s\t" % (valid_accuracy[-1]),
                    "Testacc: %s\n" % (test_accuracy[-1]),
                  "BestValidAcc: %s\n" % (best_validation_accuracy),
                  "BestTestAcc: %s\n" % (best_test_accuracy))

        return train_loss, modeler, valid_accuracy, test_accuracy

    except KeyboardInterrupt:
        print('-' * 89)
        return train_loss, modeler, valid_accuracy, test_accuracy

In [28]:
if not pointwise:
    loss_func = nn.MarginRankingLoss(margin=1, size_average=False)
else:
    loss_func = nn.BCEWithLogitsLoss()
    training_model += '_pointwise'
parameter_dict['epochs'] = 100
if False:
    train_loss, modeler, valid_accuracy, test_accuracy  = cc.training_loop(training_model = training_model,
                       parameter_dict = parameter_dict,
                       modeler = modeler,
                       train_loader = train_loader,
                       optimizer=optimizer,
                       loss_func=loss_func,
                       data=data,
                       dataset=parameter_dict['dataset'],
                       device=device,
                       test_every=1,
                       validate_every=1,
                        pointwise=pointwise,
                       problem='core_chain')
    parameter_dict['epochs'] = 10
    lr_args = {'iterations': (iterations+1)*parameter_dict['epochs'], 'cut_frac': 0.1, 'ratio': 32}
    lr_schedule = lriters.LearningRateScheduler(optimizer, lr_args, lriters.SlantedTriangularLR)
    train_loss, modeler, valid_accuracy, test_accuracy = training_loop(training_model = training_model,
                                                                               parameter_dict = parameter_dict,
                                                                               modeler = modeler,
                                                                               train_loader = train_loader,
                                                                               optimizer=optimizer,
                                                                               loss_func=loss_func,
                                                                               data=data,
                                                                               dataset=parameter_dict['dataset'],
                                                                               device=device,
                                                                               test_every=1,
                                                                               validate_every=1,
                                                                               lrschedule = lr_schedule,
                                                                               pointwise=pointwise,
                                                                               problem='core_chain')

/data/priyansh/conda/anaconda3/envs/fastai/lib/python3.6/site-packages/torch/nn/functional.py:52: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


In [29]:
# parameter_dict['epochs'] = 1
# optimizer.param_groups[-1]['lr'] = 0.001
# lr_args = {'iterations': (iterations+1)*parameter_dict['epochs'], 'cut_frac': 0.1, 'ratio': 32}
# lr_schedule = lriters.LearningRateScheduler(optimizer, lr_args, lriters.SlantedTriangularLR)
# train_loss, modeler, valid_accuracy, test_accuracy = training_loop(training_model = training_model,
#                                                                            parameter_dict = parameter_dict,
#                                                                            modeler = modeler,
#                                                                            train_loader = train_loader,
#                                                                            optimizer=optimizer,
#                                                                            loss_func=loss_func,
#                                                                            data=data,
#                                                                            dataset=parameter_dict['dataset'],
#                                                                            device=device,
#                                                                            test_every=1,
#                                                                            validate_every=1,
#                                                                            lrschedule = lr_schedule,
#                                                                            pointwise=pointwise,
#                                                                            problem='core_chain')

In [30]:
parameter_dict['epochs'] = 1
optimizer.param_groups[-1]['lr'] = 0.001
optimizer.param_groups[-2]['lr'] = 0.001
# # optimizer.param_groups[-3]['lr'] = 0.001
lr_args = {'iterations': (iterations+1)*parameter_dict['epochs'], 'cut_frac': 0.1, 'ratio': 32}
lr_schedule = lriters.LearningRateScheduler(optimizer, lr_args, lriters.SlantedTriangularLR)
train_loss, modeler, valid_accuracy, test_accuracy = training_loop(training_model = training_model,
                                                                           parameter_dict = parameter_dict,
                                                                           modeler = modeler,
                                                                           train_loader = train_loader,
                                                                           optimizer=optimizer,
                                                                           loss_func=loss_func,
                                                                           data=data,
                                                                           dataset=parameter_dict['dataset'],
                                                                           device=device,
                                                                           test_every=1,
                                                                           validate_every=1,
                                                                           lrschedule = lr_schedule,
                                                                           pointwise=pointwise,
                                                                           problem='core_chain')

the dataset is  lcquad
Epoch:  0 / 1
Batch:	0 /1000	:  1.3254733085632324 	1.3273024559020996 	830.1444702148438
Batch:	1 /1000	:  1.2690496444702148 	2.598003387451172 	835.0595092773438
Batch:	2 /1000	:  1.2916851043701172 	3.8932571411132812 	774.1123657226562
Batch:	3 /1000	:  1.255953311920166 	5.152727842330933 	764.3779907226562
Batch:	4 /1000	:  1.2722795009613037 	6.428865194320679 	704.6858520507812
Batch:	5 /1000	:  1.2793288230895996 	7.712567567825317 	714.433837890625
Batch:	6 /1000	:  1.2794723510742188 	8.995824813842773 	737.313720703125
Batch:	7 /1000	:  1.2573659420013428 	10.255369663238525 	759.1644287109375
Batch:	8 /1000	:  1.3024368286132812 	11.56149959564209 	687.2119140625
Batch:	9 /1000	:  1.2269296646118164 	12.791270017623901 	591.9115600585938
Batch:	10 /1000	:  1.2229669094085693 	14.017300844192505 	638.962646484375
Batch:	11 /1000	:  1.250309944152832 	15.270742177963257 	625.64453125
Batch:	12 /1000	:  1.300203800201416 	16.573471307754517 	572.380920

Batch:	107 /1000	:  1.2088193893432617 	137.83324003219604 	219.7392120361328
Batch:	108 /1000	:  1.2583205699920654 	139.09351992607117 	111.71745300292969
Batch:	109 /1000	:  1.2311723232269287 	140.32805228233337 	237.8780975341797
Batch:	110 /1000	:  1.2351405620574951 	141.56672430038452 	226.6460723876953
Batch:	111 /1000	:  1.2263269424438477 	142.79599714279175 	236.49876403808594
Batch:	112 /1000	:  1.2222514152526855 	144.02058506011963 	117.33279418945312
Batch:	113 /1000	:  1.2515709400177002 	145.27473640441895 	349.4302673339844
Batch:	114 /1000	:  1.2413508892059326 	146.51985216140747 	200.56932067871094
Batch:	115 /1000	:  1.2256076335906982 	147.74736714363098 	218.74627685546875
Batch:	116 /1000	:  1.2228388786315918 	148.97263026237488 	111.97452545166016
Batch:	117 /1000	:  1.2202727794647217 	150.19547319412231 	219.41783142089844
Batch:	118 /1000	:  1.2478084564208984 	151.44628524780273 	105.98619842529297
Batch:	119 /1000	:  1.2198536396026611 	152.668704032897

Batch:	213 /1000	:  1.3057949542999268 	271.9036936759949 	187.87380981445312
Batch:	214 /1000	:  1.2480287551879883 	273.1543414592743 	169.5994110107422
Batch:	215 /1000	:  1.3140714168548584 	274.47149419784546 	143.98753356933594
Batch:	216 /1000	:  1.2913830280303955 	275.76522850990295 	149.7061309814453
Batch:	217 /1000	:  1.2659626007080078 	277.03377890586853 	156.33795166015625
Batch:	218 /1000	:  1.2534136772155762 	278.2912323474884 	153.70484924316406
Batch:	219 /1000	:  1.298574686050415 	279.5953788757324 	185.77410888671875
Batch:	220 /1000	:  1.3141889572143555 	280.91210651397705 	185.8035430908203
Batch:	221 /1000	:  1.237830400466919 	282.15274000167847 	209.99913024902344
Batch:	222 /1000	:  1.2400364875793457 	283.3967354297638 	148.52593994140625
Batch:	223 /1000	:  1.2321040630340576 	284.63081431388855 	181.94430541992188
Batch:	224 /1000	:  1.2534279823303223 	285.8879632949829 	194.310791015625
Batch:	225 /1000	:  1.2760980129241943 	287.1660807132721 	158.75

Batch:	319 /1000	:  1.2877740859985352 	406.31154561042786 	158.9256591796875
Batch:	320 /1000	:  1.237912893295288 	407.55156803131104 	147.32028198242188
Batch:	321 /1000	:  1.2433831691741943 	408.7970733642578 	122.38286590576172
Batch:	322 /1000	:  1.3179187774658203 	410.117041349411 	179.34715270996094
Batch:	323 /1000	:  1.2787554264068604 	411.400333404541 	235.61306762695312
Batch:	324 /1000	:  1.2850277423858643 	412.68797874450684 	117.1728515625
Batch:	325 /1000	:  1.2198264598846436 	413.90974521636963 	136.54010009765625
Batch:	326 /1000	:  1.2514033317565918 	415.16416454315186 	139.69049072265625
Batch:	327 /1000	:  1.2637276649475098 	416.43113136291504 	161.8275604248047
Batch:	328 /1000	:  1.2759840488433838 	417.70920419692993 	148.8406524658203
Batch:	329 /1000	:  1.236203670501709 	418.94736218452454 	149.93507385253906
Batch:	330 /1000	:  1.283834457397461 	420.23459935188293 	147.1107940673828
Batch:	331 /1000	:  1.198068618774414 	421.4355025291443 	141.012237

vocab = ei.vocab
inv_vocab = {v:k for k,v in vocab.items()}
inv_vocab[1]

## Number of ques with unk

np.mean([(dt[0][0] == 1).any() for dt in correct_t]), np.mean([(dt[0][0] == 1).any() for dt in incorrect_t])

## Length of questions

np.mean([len([tok.item() for tok in ques[0][0] if tok != 0]) -3 for ques in correct_t]), \
np.mean([len([tok.item() for tok in ques[0][0] if tok != 0]) -3 for ques in incorrect_t])

## Avg length of neg paths

np.mean([np.unique(ques[1], axis=0).shape[0] - 1 for ques in correct_t]), \
np.std([np.unique(ques[1], axis=0).shape[0] - 1 for ques in correct_t]), \
np.mean([np.unique(ques[1], axis=0).shape[0] - 1 for ques in incorrect_t]), \
np.std([np.unique(ques[1], axis=0).shape[0] - 1 for ques in incorrect_t])

np.mean([np.unique(ques[1], axis=0).shape[0] - 1 for ques in correct_t+incorrect_t]) 

' '.join([inv_vocab[i.item()] for i in correct_t[np.random.randint(0, len(correct_t))][0][0] if i.item() != 0])

lens = []
for j in range(len(incorrect_t)):
    l = [inv_vocab[i.item()] for i in incorrect_t[j][1][-1] if inv_vocab[i.item()] != '_pad_']
    lens.append(len([tok for tok in l if tok in ['-','+']]))
    
np.mean(lens)

In [31]:
parameter_dict['epochs'] = 1
optimizer.param_groups[-1]['lr'] = 0.001
optimizer.param_groups[-2]['lr'] = 0.001
optimizer.param_groups[-3]['lr'] = 0.001/4
# # optimizer.param_groups[-4]['lr'] = 0.001
lr_args = {'iterations': (iterations+1)*parameter_dict['epochs'], 'cut_frac': 0.1, 'ratio': 32}
lr_schedule = lriters.LearningRateScheduler(optimizer, lr_args, lriters.SlantedTriangularLR)
traces = training_loop(training_model = training_model, parameter_dict = parameter_dict, modeler = modeler,
                       train_loader = train_loader, optimizer=optimizer, loss_func=loss_func,
                       data=data, dataset=parameter_dict['dataset'], device=device,
                       test_every=1, validate_every=1, lrschedule = lr_schedule,
                       pointwise=pointwise, problem='core_chain')
train_loss.extend(traces[0])
modeler = traces[1]
valid_accuracy.extend(traces[2])
test_accuracy.extend(traces[3])

the dataset is  lcquad
Epoch:  0 / 1
Batch:	0 /1000	:  1.3130228519439697 	1.3152565956115723 	169.76248168945312
Batch:	1 /1000	:  1.3088793754577637 	2.6264073848724365 	210.7818145751953
Batch:	2 /1000	:  1.36503005027771 	3.9947288036346436 	167.50933837890625
Batch:	3 /1000	:  1.4170727729797363 	5.415192365646362 	177.86886596679688
Batch:	4 /1000	:  1.2965149879455566 	6.717931747436523 	136.97906494140625
Batch:	5 /1000	:  1.264991283416748 	7.98749041557312 	131.37367248535156
Batch:	6 /1000	:  1.228384256362915 	9.219987630844116 	182.7617645263672
Batch:	7 /1000	:  1.269336462020874 	10.49264645576477 	133.09767150878906
Batch:	8 /1000	:  1.2782232761383057 	11.774281024932861 	137.3757781982422
Batch:	9 /1000	:  1.3305797576904297 	13.109719038009644 	205.13873291015625
Batch:	10 /1000	:  1.2464425563812256 	14.363184928894043 	157.0504913330078
Batch:	11 /1000	:  1.289074182510376 	15.65751028060913 	130.4880828857422
Batch:	12 /1000	:  1.2481560707092285 	16.9100930690765

Batch:	107 /1000	:  1.3663930892944336 	138.7824468612671 	73.37664031982422
Batch:	108 /1000	:  1.1892528533935547 	139.9737253189087 	115.07461547851562
Batch:	109 /1000	:  1.267016887664795 	141.24304580688477 	104.8761978149414
Batch:	110 /1000	:  1.317322015762329 	142.56373190879822 	103.46662139892578
Batch:	111 /1000	:  1.3392810821533203 	143.9051251411438 	223.91812133789062
Batch:	112 /1000	:  1.2582404613494873 	145.16668701171875 	121.81462097167969
Batch:	113 /1000	:  1.264709234237671 	146.43430161476135 	149.20420837402344
Batch:	114 /1000	:  1.2119669914245605 	147.64913392066956 	113.8922348022461
Batch:	115 /1000	:  1.2420976161956787 	148.8933572769165 	118.94296264648438
Batch:	116 /1000	:  1.2445008754730225 	150.1410059928894 	104.49920654296875
Batch:	117 /1000	:  1.2131645679473877 	151.35615372657776 	120.99675750732422
Batch:	118 /1000	:  1.2292261123657227 	152.58879780769348 	123.41382598876953
Batch:	119 /1000	:  1.2587854862213135 	153.85081315040588 	112

Batch:	213 /1000	:  1.232776165008545 	270.52460741996765 	70.13615417480469
Batch:	214 /1000	:  1.1886634826660156 	271.7153072357178 	90.82850646972656
Batch:	215 /1000	:  1.1998882293701172 	272.9172112941742 	74.22390747070312
Batch:	216 /1000	:  1.2067406177520752 	274.12669014930725 	44.380889892578125
Batch:	217 /1000	:  1.255091905593872 	275.3843629360199 	78.77191162109375
Batch:	218 /1000	:  1.1960532665252686 	276.5821979045868 	68.89385986328125
Batch:	219 /1000	:  1.231557846069336 	277.81611728668213 	73.02942657470703
Batch:	220 /1000	:  1.1978237628936768 	279.01626658439636 	164.05230712890625
Batch:	221 /1000	:  1.2535619735717773 	280.27225399017334 	58.02042770385742
Batch:	222 /1000	:  1.2221028804779053 	281.4963071346283 	78.60413360595703
Batch:	223 /1000	:  1.2431726455688477 	282.74188137054443 	139.39048767089844
Batch:	224 /1000	:  1.2269561290740967 	283.97117042541504 	66.32203674316406
Batch:	225 /1000	:  1.2515616416931152 	285.22482562065125 	39.241744

Batch:	319 /1000	:  1.2632837295532227 	403.34694361686707 	54.04634475708008
Batch:	320 /1000	:  1.234236478805542 	404.5841715335846 	77.04402923583984
Batch:	321 /1000	:  1.2393670082092285 	405.82546520233154 	77.55242919921875
Batch:	322 /1000	:  1.262291431427002 	407.0898938179016 	86.04623413085938
Batch:	323 /1000	:  1.237581491470337 	408.32991099357605 	97.13504791259766
Batch:	324 /1000	:  1.3141822814941406 	409.6466953754425 	51.00663375854492
Batch:	325 /1000	:  1.2526133060455322 	410.90138244628906 	63.6769905090332
Batch:	326 /1000	:  1.2975947856903076 	412.201251745224 	51.439327239990234
Batch:	327 /1000	:  1.249135971069336 	413.45313334465027 	68.59125518798828
Batch:	328 /1000	:  1.3223001956939697 	414.77844190597534 	72.44987487792969
Batch:	329 /1000	:  1.2411808967590332 	416.02256083488464 	68.9510726928711
Batch:	330 /1000	:  1.2737886905670166 	417.3005385398865 	85.83570861816406
Batch:	331 /1000	:  1.2474262714385986 	418.55027079582214 	79.581268310546

In [32]:
parameter_dict['epochs'] = 1
optimizer.param_groups[-1]['lr'] = 0.001
optimizer.param_groups[-2]['lr'] = 0.001
optimizer.param_groups[-3]['lr'] = 0.001/4
optimizer.param_groups[-4]['lr'] = 0.001/8
# # optimizer.param_groups[-5]['lr'] = 0.001/8
lr_args = {'iterations': (iterations+1)*parameter_dict['epochs'], 'cut_frac': 0.1, 'ratio': 32}
lr_schedule = lriters.LearningRateScheduler(optimizer, lr_args, lriters.SlantedTriangularLR)
traces = training_loop(training_model = training_model,
                                                                           parameter_dict = parameter_dict,
                                                                           modeler = modeler,
                                                                           train_loader = train_loader,
                                                                           optimizer=optimizer,
                                                                           loss_func=loss_func,
                                                                           data=data,
                                                                           dataset=parameter_dict['dataset'],
                                                                           device=device,
                                                                           test_every=1,
                                                                           validate_every=1,
                                                                           lrschedule = lr_schedule,
                                                                           pointwise=pointwise,
                                                                           problem='core_chain')
train_loss.extend(traces[0])
modeler = traces[1]
valid_accuracy.extend(traces[2])
test_accuracy.extend(traces[3])

the dataset is  lcquad
Epoch:  0 / 1
Batch:	0 /1000	:  1.32124924659729 	1.3237755298614502 	85.83172607421875
Batch:	1 /1000	:  1.2099525928497314 	2.5361123085021973 	53.219276428222656
Batch:	2 /1000	:  1.3796770572662354 	3.9189579486846924 	66.0035629272461
Batch:	3 /1000	:  1.2483978271484375 	5.170247554779053 	86.00163269042969
Batch:	4 /1000	:  1.2681899070739746 	6.4442198276519775 	45.730003356933594
Batch:	5 /1000	:  1.2940585613250732 	7.741135120391846 	66.00973510742188
Batch:	6 /1000	:  1.2618803977966309 	9.015076637268066 	36.30858612060547
Batch:	7 /1000	:  1.2563672065734863 	10.274343490600586 	47.129295349121094
Batch:	8 /1000	:  1.4480023384094238 	11.726442575454712 	73.3488540649414
Batch:	9 /1000	:  1.2305078506469727 	12.96298861503601 	60.66656494140625
Batch:	10 /1000	:  1.2484431266784668 	14.216046333312988 	56.88569259643555
Batch:	11 /1000	:  1.2431275844573975 	15.463495969772339 	51.397377014160156
Batch:	12 /1000	:  1.2299895286560059 	16.69737529754

Batch:	108 /1000	:  1.3858082294464111 	141.3586003780365 	34.54985809326172
Batch:	109 /1000	:  1.2461826801300049 	142.60799074172974 	53.62181854248047
Batch:	110 /1000	:  1.2473218441009521 	143.85859942436218 	42.3884391784668
Batch:	111 /1000	:  1.2425265312194824 	145.10632491111755 	40.9425048828125
Batch:	112 /1000	:  1.349247932434082 	146.45897912979126 	44.1781005859375
Batch:	113 /1000	:  1.2709026336669922 	147.73470973968506 	57.27922439575195
Batch:	114 /1000	:  1.2825522422790527 	149.0208020210266 	57.60369110107422
Batch:	115 /1000	:  1.2395060062408447 	150.26458358764648 	61.26639175415039
Batch:	116 /1000	:  1.2563707828521729 	151.52376055717468 	36.293968200683594
Batch:	117 /1000	:  1.2571160793304443 	152.7836639881134 	45.581390380859375
Batch:	118 /1000	:  1.221684217453003 	154.00947666168213 	55.67655944824219
Batch:	119 /1000	:  1.307356357574463 	155.32070589065552 	98.11894989013672
Batch:	120 /1000	:  1.2366087436676025 	156.56082582473755 	37.76777648

Batch:	214 /1000	:  1.2877821922302246 	281.86829948425293 	41.63950729370117
Batch:	215 /1000	:  1.2949109077453613 	283.1669819355011 	38.77716827392578
Batch:	216 /1000	:  1.2646079063415527 	284.4352340698242 	42.377281188964844
Batch:	217 /1000	:  1.3210041522979736 	285.7600860595703 	32.848873138427734
Batch:	218 /1000	:  1.2867932319641113 	287.05154848098755 	37.779727935791016
Batch:	219 /1000	:  1.2440571784973145 	288.3001630306244 	32.62299346923828
Batch:	220 /1000	:  1.2445001602172852 	289.5489134788513 	32.751319885253906
Batch:	221 /1000	:  1.2961618900299072 	290.8487899303436 	27.60083770751953
Batch:	222 /1000	:  1.328132152557373 	292.1804554462433 	32.01878356933594
Batch:	223 /1000	:  1.3025686740875244 	293.4868347644806 	48.99752426147461
Batch:	224 /1000	:  1.2725334167480469 	294.7638592720032 	47.05369186401367
Batch:	225 /1000	:  1.2618160247802734 	296.0287470817566 	37.07176971435547
Batch:	226 /1000	:  1.3055427074432373 	297.33693504333496 	25.81088829

Batch:	320 /1000	:  1.2609283924102783 	417.5379047393799 	38.686100006103516
Batch:	321 /1000	:  1.2602276802062988 	418.8014307022095 	45.46683120727539
Batch:	322 /1000	:  1.2347023487091064 	420.03981375694275 	32.68653869628906
Batch:	323 /1000	:  1.254084587097168 	421.2976932525635 	39.972991943359375
Batch:	324 /1000	:  1.3102788925170898 	422.6106860637665 	30.312267303466797
Batch:	325 /1000	:  1.259047031402588 	423.8725800514221 	25.26175880432129
Batch:	326 /1000	:  1.2510950565338135 	425.12647438049316 	30.039348602294922
Batch:	327 /1000	:  1.2904284000396729 	426.4202778339386 	24.039230346679688
Batch:	328 /1000	:  1.281191110610962 	427.70538544654846 	56.00090789794922
Batch:	329 /1000	:  1.3164045810699463 	429.0247275829315 	44.82472229003906
Batch:	330 /1000	:  1.2416231632232666 	430.27074909210205 	25.96023941040039
Batch:	331 /1000	:  1.2504487037658691 	431.52436566352844 	25.236019134521484
Batch:	332 /1000	:  1.2389678955078125 	432.7677457332611 	32.953071

In [33]:
parameter_dict['epochs'] = 1
optimizer.param_groups[-1]['lr'] = 0.001
optimizer.param_groups[-2]['lr'] = 0.001
optimizer.param_groups[-3]['lr'] = 0.001/4
optimizer.param_groups[-4]['lr'] = 0.001/8
optimizer.param_groups[-5]['lr'] = 0.001/16
# optimizer.param_groups[-6]['lr'] = 0.001/16
lr_args = {'iterations': (iterations+1)*parameter_dict['epochs'], 'cut_frac': 0.1, 'ratio': 32}
# lr_args['iterations'] = lr_args['iterations']*parameter_dict['epochs']
lr_schedule = lriters.LearningRateScheduler(optimizer, lr_args, lriters.SlantedTriangularLR)
traces = training_loop(training_model = training_model,
                                                                           parameter_dict = parameter_dict,
                                                                           modeler = modeler,
                                                                           train_loader = train_loader,
                                                                           optimizer=optimizer,
                                                                           loss_func=loss_func,
                                                                           data=data,
                                                                           dataset=parameter_dict['dataset'],
                                                                           device=device,\
                                                                           test_every=1,
                                                                           validate_every=1,
                                                                           lrschedule = lr_schedule,
                                                                           pointwise=pointwise,
                                                                           problem='core_chain')
train_loss.extend(traces[0])
modeler = traces[1]
valid_accuracy.extend(traces[2])
test_accuracy.extend(traces[3])

the dataset is  lcquad
Epoch:  0 / 1
Batch:	0 /1000	:  1.2768306732177734 	1.278825283050537 	37.85003662109375
Batch:	1 /1000	:  1.3168792724609375 	2.598295211791992 	29.321861267089844
Batch:	2 /1000	:  1.3784592151641846 	3.979626417160034 	27.948348999023438
Batch:	3 /1000	:  1.3751537799835205 	5.357321500778198 	36.25046920776367
Batch:	4 /1000	:  1.371263027191162 	6.731234550476074 	31.453453063964844
Batch:	5 /1000	:  1.3304705619812012 	8.063727617263794 	54.06328201293945
Batch:	6 /1000	:  1.366030216217041 	9.433696985244751 	43.84553146362305
Batch:	7 /1000	:  1.308361291885376 	10.743913650512695 	25.127960205078125
Batch:	8 /1000	:  1.356065273284912 	12.101914167404175 	34.42560577392578
Batch:	9 /1000	:  1.232236623764038 	13.33645486831665 	27.626056671142578
Batch:	10 /1000	:  1.2676732540130615 	14.607535123825073 	27.977275848388672
Batch:	11 /1000	:  1.2246625423431396 	15.834203243255615 	29.641468048095703
Batch:	12 /1000	:  1.2490482330322266 	17.0866949558258

Batch:	107 /1000	:  1.3353550434112549 	138.7271385192871 	25.954986572265625
Batch:	108 /1000	:  1.3481957912445068 	140.0784330368042 	21.718603134155273
Batch:	109 /1000	:  1.3654625415802002 	141.44623589515686 	48.23026657104492
Batch:	110 /1000	:  1.376492977142334 	142.82459235191345 	40.75068283081055
Batch:	111 /1000	:  1.3703508377075195 	144.1970238685608 	47.310569763183594
Batch:	112 /1000	:  1.2386829853057861 	145.43841743469238 	22.307472229003906
Batch:	113 /1000	:  1.2506744861602783 	146.69114708900452 	52.31193923950195
Batch:	114 /1000	:  1.235891580581665 	147.92951679229736 	39.66521072387695
Batch:	115 /1000	:  1.275083303451538 	149.20650482177734 	30.1549129486084
Batch:	116 /1000	:  1.25364351272583 	150.46232962608337 	37.11591339111328
Batch:	117 /1000	:  1.2338016033172607 	151.6979718208313 	45.753353118896484
Batch:	118 /1000	:  1.2391700744628906 	152.93917179107666 	16.228290557861328
Batch:	119 /1000	:  1.198146104812622 	154.14027309417725 	23.831132

Batch:	213 /1000	:  1.2653725147247314 	274.0187454223633 	15.96982192993164
Batch:	214 /1000	:  1.1602017879486084 	275.1807677745819 	22.25250244140625
Batch:	215 /1000	:  1.276085615158081 	276.4584617614746 	36.7999267578125
Batch:	216 /1000	:  1.2262349128723145 	277.6878328323364 	34.920616149902344
Batch:	217 /1000	:  1.2106363773345947 	278.90012788772583 	20.776002883911133
Batch:	218 /1000	:  1.2632291316986084 	280.1650478839874 	44.698951721191406
Batch:	219 /1000	:  1.2623343467712402 	281.42920422554016 	24.420455932617188
Batch:	220 /1000	:  1.2540245056152344 	282.685364484787 	24.32034683227539
Batch:	221 /1000	:  1.2644975185394287 	283.951979637146 	17.469144821166992
Batch:	222 /1000	:  1.240133285522461 	285.1952168941498 	17.586545944213867
Batch:	223 /1000	:  1.2363736629486084 	286.43475341796875 	26.838817596435547
Batch:	224 /1000	:  1.2755126953125 	287.71264147758484 	20.827016830444336
Batch:	225 /1000	:  1.2286360263824463 	288.9438302516937 	25.7467479705

Batch:	319 /1000	:  1.2055795192718506 	406.0114676952362 	16.871509552001953
Batch:	320 /1000	:  1.220130205154419 	407.23447918891907 	19.341365814208984
Batch:	321 /1000	:  1.2839772701263428 	408.52045798301697 	19.31908416748047
Batch:	322 /1000	:  1.2482354640960693 	409.77097725868225 	11.646470069885254
Batch:	323 /1000	:  1.2601463794708252 	411.0341739654541 	35.21086120605469
Batch:	324 /1000	:  1.2199046611785889 	412.25598883628845 	31.968170166015625
Batch:	325 /1000	:  1.2602121829986572 	413.518324136734 	34.24946975708008
Batch:	326 /1000	:  1.2728288173675537 	414.7934603691101 	28.48866081237793
Batch:	327 /1000	:  1.280620813369751 	416.076251745224 	21.021055221557617
Batch:	328 /1000	:  1.2391471862792969 	417.31741547584534 	19.63140869140625
Batch:	329 /1000	:  1.2651042938232422 	418.58521461486816 	19.685871124267578
Batch:	330 /1000	:  1.2347185611724854 	419.8223581314087 	16.121627807617188
Batch:	331 /1000	:  1.2470910549163818 	421.07136702537537 	21.4744

In [ ]:
parameter_dict['epochs'] = 35
optimizer.param_groups[-1]['lr'] = 0.001
optimizer.param_groups[-2]['lr'] = 0.001
optimizer.param_groups[-3]['lr'] = 0.001/4
optimizer.param_groups[-4]['lr'] = 0.001/8
optimizer.param_groups[-5]['lr'] = 0.001/16
optimizer.param_groups[-6]['lr'] = 0.001/16
lr_args = {'iterations': (iterations+1)*parameter_dict['epochs'], 'cut_frac': 0.1, 'ratio': 32}
# lr_args['iterations'] = lr_args['iterations']*parameter_dict['epochs']
lr_schedule = lriters.LearningRateScheduler(optimizer, lr_args, lriters.SlantedTriangularLR)
traces = training_loop(training_model = training_model,
                                                                           parameter_dict = parameter_dict,
                                                                           modeler = modeler,
                                                                           train_loader = train_loader,
                                                                           optimizer=optimizer,
                                                                           loss_func=loss_func,
                                                                           data=data,
                                                                           dataset=parameter_dict['dataset'],
                                                                           device=device,\
                                                                           test_every=1,
                                                                           validate_every=1,
                                                                           lrschedule = lr_schedule,
                                                                           pointwise=pointwise,
                                                                           problem='core_chain')
train_loss.extend(traces[0])
modeler = traces[1]
valid_accuracy.extend(traces[2])
test_accuracy.extend(traces[3])

the dataset is  lcquad
Epoch:  0 / 35
Batch:	0 /1000	:  1.2269518375396729 	1.2287406921386719 	21.994068145751953
Batch:	1 /1000	:  1.231982707977295 	2.4625067710876465 	11.258450508117676
Batch:	2 /1000	:  1.2803356647491455 	3.7462058067321777 	15.113983154296875
Batch:	3 /1000	:  1.2292585372924805 	4.97736668586731 	28.925540924072266
Batch:	4 /1000	:  1.263441801071167 	6.243621587753296 	22.421527862548828
Batch:	5 /1000	:  1.2540507316589355 	7.499629497528076 	25.54241371154785
Batch:	6 /1000	:  1.2452034950256348 	8.748214483261108 	15.045126914978027
Batch:	7 /1000	:  1.2563881874084473 	10.006280899047852 	11.226070404052734
Batch:	8 /1000	:  1.2584049701690674 	11.268072366714478 	27.747726440429688
Batch:	9 /1000	:  1.1996774673461914 	12.46968674659729 	19.61392593383789
Batch:	10 /1000	:  1.2531323432922363 	13.72523307800293 	19.387235641479492
Batch:	11 /1000	:  1.243129014968872 	14.970253944396973 	28.796772003173828
Batch:	12 /1000	:  1.2306084632873535 	16.204046

Batch:	107 /1000	:  1.2488162517547607 	134.5128903388977 	23.593242645263672
Batch:	108 /1000	:  1.2472381591796875 	135.76308131217957 	20.422073364257812
Batch:	109 /1000	:  1.2436068058013916 	137.0086534023285 	22.044702529907227
Batch:	110 /1000	:  1.2851073741912842 	138.29644536972046 	24.13651466369629
Batch:	111 /1000	:  1.213233232498169 	139.51430201530457 	18.56678581237793
Batch:	112 /1000	:  1.2022590637207031 	140.7187798023224 	33.802879333496094
Batch:	113 /1000	:  1.2790815830230713 	141.99984502792358 	20.60016441345215
Batch:	114 /1000	:  1.268904685974121 	143.27143716812134 	18.091365814208984
Batch:	115 /1000	:  1.2466652393341064 	144.52008724212646 	33.55143356323242
Batch:	116 /1000	:  1.2591540813446045 	145.7820281982422 	21.613542556762695
Batch:	117 /1000	:  1.2403128147125244 	147.0245532989502 	32.930702209472656
Batch:	118 /1000	:  1.2532317638397217 	148.28100514411926 	19.10930633544922
Batch:	119 /1000	:  1.2443320751190186 	149.52781343460083 	31.1

Batch:	212 /1000	:  1.2590162754058838 	266.62580823898315 	14.947704315185547
Batch:	213 /1000	:  1.2758471965789795 	267.90385460853577 	14.648153305053711
Batch:	214 /1000	:  1.2238059043884277 	269.1303141117096 	18.720949172973633
Batch:	215 /1000	:  1.248896598815918 	270.38130402565 	6.723738193511963
Batch:	216 /1000	:  1.2386221885681152 	271.6220054626465 	29.720792770385742
Batch:	217 /1000	:  1.2053396701812744 	272.82915329933167 	26.782827377319336
Batch:	218 /1000	:  1.281172513961792 	274.11213541030884 	13.257681846618652
Batch:	219 /1000	:  1.2799618244171143 	275.39500999450684 	18.58748435974121
Batch:	220 /1000	:  1.2379226684570312 	276.634948015213 	16.308025360107422
Batch:	221 /1000	:  1.249072790145874 	277.8861939907074 	24.17953109741211
Batch:	222 /1000	:  1.2688531875610352 	279.15712547302246 	19.279685974121094
Batch:	223 /1000	:  1.2670035362243652 	280.42637872695923 	15.404142379760742
Batch:	224 /1000	:  1.2300562858581543 	281.65884232521057 	12.361

Batch:	318 /1000	:  1.2624506950378418 	400.188898563385 	30.412891387939453
Batch:	319 /1000	:  1.2432878017425537 	401.4351966381073 	16.463808059692383
Batch:	320 /1000	:  1.2677335739135742 	402.70466709136963 	21.35449981689453
Batch:	321 /1000	:  1.2202208042144775 	403.92641162872314 	11.520313262939453
Batch:	322 /1000	:  1.2635743618011475 	405.1926190853119 	22.48514747619629
Batch:	323 /1000	:  1.2060503959655762 	406.400887966156 	14.351123809814453
Batch:	324 /1000	:  1.2325003147125244 	407.6365051269531 	13.53794002532959
Batch:	325 /1000	:  1.2551007270812988 	408.8936140537262 	27.261838912963867
Batch:	326 /1000	:  1.2126944065093994 	410.10861921310425 	7.446764945983887
Batch:	327 /1000	:  1.3001186847686768 	411.41083121299744 	12.584754943847656
Batch:	328 /1000	:  1.278245210647583 	412.69389629364014 	21.44231414794922
Batch:	329 /1000	:  1.3046984672546387 	414.00057721138 	31.725662231445312
Batch:	330 /1000	:  1.221743106842041 	415.2249014377594 	19.41164016

Batch:	71 /1000	:  1.2676308155059814 	89.67442893981934 	13.531698226928711
Batch:	72 /1000	:  1.2324228286743164 	90.90908360481262 	21.502769470214844
Batch:	73 /1000	:  1.2471234798431396 	92.15834760665894 	25.71760368347168
Batch:	74 /1000	:  1.253075361251831 	93.41354775428772 	9.660490036010742
Batch:	75 /1000	:  1.251573085784912 	94.66710877418518 	31.962438583374023
Batch:	76 /1000	:  1.2116074562072754 	95.8818211555481 	29.419418334960938
Batch:	77 /1000	:  1.2555382251739502 	97.13946795463562 	17.271743774414062
Batch:	78 /1000	:  1.2696428298950195 	98.41240763664246 	22.57701873779297
Batch:	79 /1000	:  1.2351891994476318 	99.64932942390442 	15.760475158691406
Batch:	80 /1000	:  1.2796556949615479 	100.93213438987732 	20.386810302734375
Batch:	81 /1000	:  1.2382659912109375 	102.17242383956909 	15.33205795288086
Batch:	82 /1000	:  1.234161615371704 	103.40872287750244 	12.138309478759766
Batch:	83 /1000	:  1.2735228538513184 	104.68497776985168 	22.54067611694336
Batc

...

Batch:	112 /1000	:  1.267005205154419 	145.79765057563782 	4.65317440032959
Batch:	113 /1000	:  1.257840871810913 	147.05834126472473 	0.0
Batch:	114 /1000	:  1.2457880973815918 	148.30749320983887 	4.795351982116699
Batch:	115 /1000	:  1.2556607723236084 	149.56526279449463 	1.843587875366211
Batch:	116 /1000	:  1.2604923248291016 	150.82801699638367 	0.13635635375976562
Batch:	117 /1000	:  1.2525286674499512 	152.08329916000366 	0.0
Batch:	118 /1000	:  1.2405118942260742 	153.32786750793457 	0.6099605560302734
Batch:	119 /1000	:  1.2492892742156982 	154.57993984222412 	1.6310367584228516
Batch:	120 /1000	:  1.2511401176452637 	155.83377504348755 	0.0
Batch:	121 /1000	:  1.2504522800445557 	157.08701610565186 	0.07364511489868164
Batch:	122 /1000	:  1.229954719543457 	158.31962180137634 	5.201163291931152
Batch:	123 /1000	:  1.2122790813446045 	159.53441286087036 	1.4746336936950684
Batch:	124 /1000	:  1.2058756351470947 	160.74282813072205 	0.0
Batch:	125 /1000	:  1.2340524196624756 

Batch:	223 /1000	:  1.2312994003295898 	285.16179966926575 	3.195903778076172
Batch:	224 /1000	:  1.243957281112671 	286.408390045166 	2.1314539909362793
Batch:	225 /1000	:  1.2430131435394287 	287.65565967559814 	3.90751314163208
Batch:	226 /1000	:  1.2150969505310059 	288.8735933303833 	0.0
Batch:	227 /1000	:  1.2480857372283936 	290.12597370147705 	1.432295799255371
Batch:	228 /1000	:  1.2462725639343262 	291.3746693134308 	4.042043685913086
Batch:	229 /1000	:  1.2374517917633057 	292.61647939682007 	2.511052131652832
Batch:	230 /1000	:  1.2352571487426758 	293.8557839393616 	0.0
Batch:	231 /1000	:  1.4028871059417725 	295.26311206817627 	3.0919246673583984
Batch:	232 /1000	:  1.368652105331421 	296.6344609260559 	2.2545523643493652
Batch:	233 /1000	:  1.3805148601531982 	298.01965737342834 	4.374666690826416
Batch:	234 /1000	:  1.3824849128723145 	299.40477561950684 	5.03663444519043
Batch:	235 /1000	:  1.3740215301513672 	300.783047914505 	1.2789936065673828
Batch:	236 /1000	:  1.

Batch:	334 /1000	:  1.2345061302185059 	425.820365190506 	0.6659150123596191
Batch:	335 /1000	:  1.2393274307250977 	427.06427478790283 	0.7288422584533691
Batch:	336 /1000	:  1.2170045375823975 	428.2844879627228 	0.5234088897705078
Batch:	337 /1000	:  1.2704508304595947 	429.5589644908905 	0.9251289367675781
Batch:	338 /1000	:  1.2419993877410889 	430.80365014076233 	2.443662405014038
Batch:	339 /1000	:  1.2468883991241455 	432.05505299568176 	0.0
Batch:	340 /1000	:  1.2714660167694092 	433.32880568504333 	0.6068899631500244
Batch:	341 /1000	:  1.2216618061065674 	434.5535283088684 	2.2697582244873047
Batch:	342 /1000	:  1.2450330257415771 	435.80078172683716 	0.0
Batch:	343 /1000	:  1.2586872577667236 	437.0626895427704 	3.6340885162353516
Batch:	344 /1000	:  1.2285256385803223 	438.2945032119751 	1.450155258178711
Batch:	345 /1000	:  1.2692594528198242 	439.5664277076721 	0.732079029083252
Batch:	346 /1000	:  1.2707116603851318 	440.8405077457428 	4.628571510314941
Batch:	347 /1000

Batch:	94 /1000	:  1.2490053176879883 	120.21627044677734 	0.07780122756958008
Batch:	95 /1000	:  1.2488923072814941 	121.46786713600159 	0.05269336700439453
Batch:	96 /1000	:  1.260899305343628 	122.7318286895752 	2.2910828590393066
Batch:	97 /1000	:  1.276759147644043 	124.01180577278137 	0.4183797836303711
Batch:	98 /1000	:  1.22414231300354 	125.2390456199646 	0.3955349922180176
Batch:	99 /1000	:  1.2319841384887695 	126.47532868385315 	0.0011572837829589844
Batch:	100 /1000	:  1.220757246017456 	127.69881105422974 	0.0
Batch:	101 /1000	:  1.2843458652496338 	128.98589086532593 	0.958460807800293
Batch:	102 /1000	:  1.260608196258545 	130.25013756752014 	1.3239293098449707
Batch:	103 /1000	:  1.2331311702728271 	131.48594784736633 	0.5777268409729004
Batch:	104 /1000	:  1.2436017990112305 	132.73312878608704 	0.0
Batch:	105 /1000	:  1.2391338348388672 	133.97528648376465 	1.944007396697998
Batch:	106 /1000	:  1.2440762519836426 	135.22280311584473 	0.0
Batch:	107 /1000	:  1.2636828

Batch:	203 /1000	:  1.2565562725067139 	258.8464503288269 	0.0
Batch:	204 /1000	:  1.2184820175170898 	260.06764793395996 	7.622343063354492
Batch:	205 /1000	:  1.264939785003662 	261.3368728160858 	0.0
Batch:	206 /1000	:  1.2213916778564453 	262.56098556518555 	7.8101677894592285
Batch:	207 /1000	:  1.2161550521850586 	263.7814025878906 	0.19986820220947266
Batch:	208 /1000	:  1.2584443092346191 	265.0424180030823 	1.302872657775879
Batch:	209 /1000	:  1.2380740642547607 	266.2841718196869 	2.3292837142944336
Batch:	210 /1000	:  1.2565884590148926 	267.5431981086731 	0.5165309906005859
Batch:	211 /1000	:  1.2551052570343018 	268.8016972541809 	0.0
Batch:	212 /1000	:  1.2604646682739258 	270.0653681755066 	0.47638702392578125
Batch:	213 /1000	:  1.2674152851104736 	271.33705139160156 	2.144089698791504
Batch:	214 /1000	:  1.2264034748077393 	272.5661060810089 	5.566781997680664
Batch:	215 /1000	:  1.2925164699554443 	273.8626160621643 	2.64768123626709
Batch:	216 /1000	:  1.27353954315

In [ ]:
plt.xkcd()
plt.plot(train_loss)
plt.show()

In [ ]:
%pylab inline
pylab.rcParams['figure.figsize'] = (16, 8)

In [ ]:
plt.xkcd()
fig = plt.figure()
ax = fig.add_axes((0.1, 0.2, 0.8, 0.7))
ax.spines['right'].set_color('none')
ax.spines['top'].set_color('none')
plt.xticks([])
# plt.yticks([])
plt.plot(valid_accuracy, label="val")
# plt.show()
plt.plot(test_accuracy, label="test")
plt.gca().legend()
plt.ylabel("Accuracy")
plt.xlabel("Inevitable movement of Iterations")
fig.text(
    0.5, 0.05,
    "I gage if 't be true the accuracy doesn't immprove i shall heave the gorge on thy livings, naughty mushrump!",
    ha='center')
plt.show()
print(valid_accuracy)
print(test_accuracy)
print("validation accuracy is , ", max(valid_accuracy))
print("maximum test accuracy is , ", max(test_accuracy))
print("correct test accuracy i.e test accuracy where validation is highest is ", test_accuracy[valid_accuracy.index(max(valid_accuracy))])

In [ ]:
lr_args = {'iterations': iterations+1, 'cut_frac': 0.1, 'ratio': 32}
lr_schedule = lriters.LearningRateScheduler(optimizer, lr_args, lriters.SlantedTriangularLR)
plt.plot([lr_schedule.get() for i in range(iterations)])
plt.show()

In [ ]:
modeler.encoder_q.linear

In [ ]:
a = [ l for l in modeler.encoder.layers
c = [l for l in modeler.encoder_path.layers]
b = a + c
print(a)
b = torch.nn.ModuleList(a+c)
print(b)